In [31]:
import sys, os, re, datetime
import numpy as np
import pandas as pd

#%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator, AutoMinorLocator
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False

from functools import reduce
from itertools import product
from cycler import cycler
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import jenkspy

%config Completer.use_jedi = False

### How to install Hangul nanum font in this Docker Ubuntu OS.
Run belows in the command line

- sudo apt-get install -y fonts-nanum
- sudo fc-cache -fv

In [2]:
print ('버전: ', mpl.__version__)
print ('설치 위치: ', mpl.__file__)
print ('설정 위치: ', mpl.get_configdir())
print ('캐시 위치: ', mpl.get_cachedir())

nanum_font_list = fm.findSystemFonts(fontpaths='/usr/share/fonts/truetype/nanum/')

for font in nanum_font_list:
    print(f"Name: {fm.FontProperties(fname=font).get_name()} | Path: {font}")

버전:  3.3.4
설치 위치:  /usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py
설정 위치:  /home/hahnyi/.config/matplotlib
캐시 위치:  /home/hahnyi/.cache/matplotlib
Name: NanumMyeongjo | Path: /usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf
Name: NanumGothic | Path: /usr/share/fonts/truetype/nanum/NanumGothic.ttf
Name: NanumSquareRound | Path: /usr/share/fonts/truetype/nanum/NanumSquareRoundB.ttf
Name: NanumGothic | Path: /usr/share/fonts/truetype/nanum/NanumGothicBold.ttf
Name: NanumBarunGothic | Path: /usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf
Name: NanumMyeongjo | Path: /usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf
Name: NanumBarunGothic | Path: /usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf
Name: NanumSquareRound | Path: /usr/share/fonts/truetype/nanum/NanumSquareRoundR.ttf
Name: NanumSquare | Path: /usr/share/fonts/truetype/nanum/NanumSquareR.ttf
Name: NanumSquare | Path: /usr/share/fonts/truetype/nanum/NanumSquareB.ttf


In [3]:
fm.get_fontconfig_fonts()
font_path = "/usr/share/fonts/truetype/nanum/NanumMyeongjo.ttf"
font_prop = fm.FontProperties(fname=font_path)

In [4]:
%run "common.ipynb"

(5328, 21)
(1811,)
Index(['person_id', 'gender_concept_id', 'year_of_birth', 'month_of_birth',
       'day_of_birth', 'birth_datetime', 'race_concept_id',
       'ethnicity_concept_id', 'location_id', 'provider_id', 'care_site_id',
       'person_source_value', 'gender_source_value',
       'gender_source_concept_id', 'race_source_value',
       'race_source_concept_id', 'ethnicity_source_value',
       'ethnicity_source_concept_id', 'obfl', 'matchfl', 'one_on_two'],
      dtype='object')
code_full(drug_code_full.csv):  (5468, 6)
ccode_cancer(cancer_treatment_drug.csv):  (485, 10)
code_add(drug_code_add.csv):  (172, 10)
code_full(drug_code_full.csv):  (5468, 6)
ccode_cancer(cancer_treatment_drug.csv):  (485, 10)
code_add(drug_code_add.csv):  (172, 10)


#### 수술코드 정리

In [5]:
data_home_path = "/workspace/data/"

In [6]:
op_map = pd.read_excel(os.path.join(data_home_path, 
                                    "diagnose_operation_concept_id_mapping_list20210330_from_SeyoungSeo.xlsx"), 
                       sheet_name='수술')

In [7]:
# including only operation containing keywords
op_map = op_map[op_map['수술코드명'].str.contains(
                'tomy|operation|excision|surgery|resection|Operation|Excision|Surgery|Resection') &
               ~op_map['수술코드명'].str.contains('scop|scan|monitor|Stereo|computerized|graph|biopsy|gram|radiosurgery')]
op_map = op_map.drop_duplicates(subset='concept_id', keep='first')

#### Merging 수술코드 with procedure Dataframe 

In [8]:
procedure_df = pd.merge(procedure_pd, op_map, how='left', 
                       left_on='procedure_concept_id',
                       right_on='concept_id')
# concept_id가 Null 인 것은 모두 제외한다. 우리가 선택한 keywords를 가진 수술만 concept_id를 가진다. 
#그림을 그리는데 불필요한 procedure 정보가 들어와 가로축의 범위를 넓혀 그림을 이해하기 어렵게 만든다. 
procedure_df = procedure_df[procedure_df['concept_id'].notnull()]
print(procedure_df.columns)
procedure_df.shape

Index(['procedure_occurrence_id', 'person_id', 'procedure_concept_id',
       'procedure_date', 'procedure_datetime', 'procedure_type_concept_id',
       'modifier_concept_id', 'quantity', 'provider_id', 'visit_occurrence_id',
       'visit_detail_id', 'procedure_source_value',
       'procedure_source_concept_id', 'modifier_source_value', '수술코드', '수술코드명',
       'concept_id', 'concept_name', 'domain_id', 'vocabulary_id',
       'concept_class_id', 'standard_concept', 'concept_code'],
      dtype='object')


(13967, 23)

#### 병리검사 코드 정리 

In [9]:
# including only procedures containing keywords
amis2_proc = pd.read_excel(os.path.join(data_home_path, 
                                       "AMIS3.0_procedure_code_chart_ASIS-TOBE_code_5-3.xlsx"),
                           sheet_name='5-3', skiprows=3)
amis2_proc = amis2_proc[amis2_proc['2.0 처방코드 한글명'].str.contains('병리', regex=False, na=False) &
                       amis2_proc['2.0 처방코드 한글명'].str.contains('암조직', regex=False, na=False) &
                       ~amis2_proc['2.0 처방코드 한글명'].str.contains('침', regex=False, na=False) &
                       ~amis2_proc['2.0 처방코드 한글명'].str.contains('면역효소조직화학', regex=False, na=False)]
# 처방코드가 중복된 것들은 모두 삭제
amis2_proc = amis2_proc.drop_duplicates(subset='2.0 처방코드', keep='first')
amis2_proc['2.0 처방코드 한글명'].head(40)


4711                        병리, 부신암조직검사
4712    병리, 부신암조직검사(림프절 20개이상,블럭 40개이상)
4784     병리, 총담관암조직(림프절 20개이상,블럭 40개이상)
4790                      병리, 간외담관암조직검사
4812                         병리, 골암조직검사
4824                     병리, 척추골적출암조직검사
4829               병리, 골암조직검사(조직구축학적검사)
4854                    병리, 유방암조직검사(좌측)
4866                        병리, 유방암조직검사
4867    병리, 유방암조직검사(림프절 20개이상,블럭 40개이상)
4869                    병리, 유방암조직검사(우측)
4946                        병리, 식도암조직검사
4978                        병리, 동공암조직검사
5004                        병리, 난관암조직검사
5005             병리, 난관암조직검사(림프절 20개이상)
5018                        병리, 담낭암조직검사
5045                        병리, 인두암조직검사
5120                        병리, 신장암조직검사
5144                        병리, 대장암조직검사
5172                        병리, 후두암조직검사
5197                         병리, 간암조직검사
5209                         병리, 폐암조직검사
5330                       병리, 두경부암조직검사
5331            병리, 두경부암조직검사(림프절 20개이상)
5354                        병리, 구강암조직검사


In [10]:
patho_df = pd.merge(procedure_pd, amis2_proc, how='left', 
                       left_on='procedure_source_value',
                       right_on='2.0 처방코드')
# concept_id가 Null 인 것은 모두 제외한다. 우리가 선택한 keywords를 가진 병리조직검사만 2.0 처방코드를 가진다. 
#그림을 그리는데 불필요한 procedure 정보가 들어와 가로축의 범위를 넓혀 그림을 이해하기 어렵게 만든다.
patho_df = patho_df[patho_df['2.0 처방코드'].notnull()]
print(patho_df.columns)
patho_df.shape

Index(['procedure_occurrence_id', 'person_id', 'procedure_concept_id',
       'procedure_date', 'procedure_datetime', 'procedure_type_concept_id',
       'modifier_concept_id', 'quantity', 'provider_id', 'visit_occurrence_id',
       'visit_detail_id', 'procedure_source_value',
       'procedure_source_concept_id', 'modifier_source_value', '행위\n차수',
       '2.0 테이블명', '2.0 처방코드', '2.0 처방코드 영문명', '2.0 처방코드 한글명', '2.0 연결\n물품코드',
       '3.0\n기준정보', '3.0 대체코드', '3.0 대체코드 영문명', '3.0 대체코드 한글명',
       '3.0\n대체코드\n유효성', '3.0 대체행위코드 종료사유'],
      dtype='object')


(2377, 26)

#### 처방 히스토리 파일과 화학적항암제 조인

In [53]:
drug_df = pd.merge(drug_pd.drop_duplicates(), chemo_drug.drop_duplicates(), 
                   how='inner', left_on=['drug_source_value'], right_on=['code'])
# methotrexate tab [2.5mg] 는 항암제가 아니다. 
drug_df = drug_df[~drug_df['name'].str.contains('methotrexate tab \[2.5mg\]')]
# 화학항암제가 아닌 것은 code가 Null이고 code가 null인 것은 그림을 그리는데 불필요한 자료
drug_df = drug_df[drug_df['code'].notnull()]
pd.Series(drug_df['name'].unique()).sort_values().to_csv('drug_raw.csv')
drug_df.shape

(144815, 25)

#### 1) 임상약 제거,  2) 문자 양끝 1 공백 제거, 3) 소문자로 전환, 4) 대괄호와 그 내용 제거, 5) 약용량 제거, 6) tab inj cap 키워드 제거, 7) 희석액이라는 낱말 제거, 8) dbl+공백 제거, 9) (yuhan) kyowa 제거 , 10) 오탈자 약이름 정정, 11) 낱말 양 끝단의 임의의 개수 공백 제거, 12) 사이 공백을 underbar로 치환

In [54]:
drug_df.loc[:, 'name'] = drug_df['name']\
                            .apply(lambda x: re.sub(r'\(임상약\)|\(\?+\)', '', x.strip('\n')))
drug_df.loc[:, 'name'] = drug_df['name'].str.strip()
drug_df.loc[:, 'name'] = drug_df['name'].str.lower()
drug_df.loc[:, 'name'] = drug_df['name'].str.replace(r'\[[^]]*\]','')
drug_df.loc[:, 'name'] = drug_df['name'].str.replace(r'[0-9]+mg','')
drug_df.loc[:, 'name'] = drug_df['name']\
                             .apply(lambda x: re.sub(r'tab\.*|inj\.*|cap\.*', '', x.strip('')))
drug_df.loc[:, 'name'] = drug_df['name'].str.replace('diluent for ','')
drug_df.loc[:, 'name'] = drug_df['name'].str.replace('dbl ','')
drug_df.loc[:, 'name'] = drug_df['name'].str.replace('\(yuhan\)|kyowa','')
drug_df.loc[:, 'name'] = drug_df['name'].str.replace('evrolimus', 'everolimus')
drug_df.loc[:, 'name'] = drug_df['name']\
                             .apply(lambda x: re.sub(r'^ *| *$', '', x.strip('')))
drug_df.loc[:, 'name'] = drug_df['name'].str.replace(' ', '_')
#drug_df['name'].value_counts().sort_index().to_csv("drug_short.csv")

In [13]:
# inj이라는 키워드가 붙은 낱말이 cap이나 tab을 가지고 있는 약제를 찾아보자 
# injs = drug_df[drug_df['name'].str.contains('inj')]['name'].value_counts().index.to_numpy()
# injs_list = set([re.sub(r'inj\.*', '', x.strip()) for x in injs])
# injs_list

# tabs = np.sort(drug_df[drug_df['name'].str.contains('cap|tab')]['name'].unique())
# tabs_list = set([re.sub(r'cap\.*|tab\.*', '', x.strip()) for x in tabs])
# tabs_list

# tabs_list.intersection(injs_list)

#### 15가지 실험군에 포함된 암종의 분류

In [14]:
def enum(s) :
    bgn, ter = re.search(r'C([0-9]+)-C([0-9]+)', s).groups()
    return ",".join(['C'+str(x).zfill(2) for x in range(int(bgn), int(ter)+1)])

In [15]:
def digitize(x) :
    if re.search(r',', x) :
        cancers = x.split(",")
        cancer_enum = []
        for cancer in cancers :
            if re.search(r'-', cancer) :
                cancer_enum.append(enum(cancer))
            else :
                cancer_enum.append(cancer)
        return ",".join(cancer_enum)
    elif re.search(r'-', x) :
        return enum(x)
    else :
        return x

In [16]:
cancer_type = pd.read_csv("cancer_types.tsv", sep="\t")
cancer_type['enum'] = cancer_type['표준진단코드'].apply(digitize)
cancer_type

,코딩값,암종,cancer,표준진단코드,enum
0,1,대장직장암,colorectal,C17-C20,"C17,C18,C19,C20"
1,2,담췌암,biliary_pancreatic,C23-C25,"C23,C24,C25"
2,3,기타암,others,"C21,C26,C37-C39,C45,C47-C48,C60,C62-C63,C69,C7...","C21,C26,C37,C38,C39,C45,C47,C48,C60,C62,C63,C6..."
3,4,부인과암,gynecological,C51-C58,"C51,C52,C53,C54,C55,C56,C57,C58"
4,5,피부암,skin,C43-C44,"C43,C44"
5,6,간암,liver,C22,C22
6,7,폐암,lung,C34,C34
7,8,뇌종양,brain,C70-C72,"C70,C71,C72"
8,9,두경부암,head_n_neck,"C00-C14,C30-C33","C00,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10,C1..."
9,10,위암,stomach,C16,C16


In [17]:
def is_long_interval(intervals_drug, allow_regimen_interval=90) :
    
    # Are there any intervals greater thane allow_interval
    # when a drug is administered
    indices_other_interval = []
    for i in range(len(intervals_drug)) :
        if intervals_drug[i] > allow_regimen_interval :
            indices_other_interval.append(i)
    return indices_other_interval

In [18]:
def zero_runs(a):
    # Create an array that is 1 where a is 0, and pad each end with an extra 0.
    iszero = np.concatenate(([0], np.equal(a, 0).view(np.int8), [0]))
    absdiff = np.abs(np.diff(iszero))
    # Runs start and end where absdiff is 1.
    ranges = np.where(absdiff == 1)[0].reshape(-1, 2)
    return ranges

In [19]:
def drop_tooshort_timespan_for_btw_regimens(run_zero_ranges, allow_btw_regimens=10) :
    
    # 이 함수는 i번째 regimen과 i+1번째 regimen의 간격이 allow_btw_regimens일 미만이라면
    # 항암제가 바뀌어도 다른 regimen으로 구분하지 않고 allow_btw_regimens 이상으로 차이가 날 
    # 경우만 별도의 regimen으로 파악하는 루틴
    run_zero_ranges_rule_tight = []
    for i, run_zero_range in enumerate(run_zero_ranges) :
        if run_zero_range[1] - run_zero_range[0] >= allow_btw_regimens :
            run_zero_ranges_rule_tight.append(run_zero_range)
    return np.asarray(run_zero_ranges_rule_tight)

In [20]:
def set_representative_cancertype_in_case_of_others(pid_condition) :
    
    # 환자의 대표 암종이 others 일 경우 같은 날에 진단 받은 others가 아닌 다른 암종이 있을 때
    # others가 아닌 cancer type이 대표 암종으로 선택

    # others를 최초를 선고 받은 날을 찾고 
    pid_others_diag_date = pid_condition[pid_condition['cancer_type']==\
                                        'others'].loc[0, 'condition_start_date']
    # 최초 진단일에 진단받은 암종이 최소 2개 이상인가?
    if pid_condition[pid_condition['condition_start_date']==\
                     pid_others_diag_date].shape[0] > 1 :
        # 최초진단일에 진단받은 암종을 모두 numpy array 저장
        representative_pid_cancertype = pid_condition[pid_condition['condition_start_date']==\
                                                     pid_others_diag_date]\
                                                    .loc[:, 'cancer_type'].values
        # 암종이 others가 아닌 것은 모두 array로 추출
        pid_cancertypes = representative_pid_cancertype[np.isin(representative_pid_cancertype, 
                                                      'others', invert=True)]                    
        # others가 아닌 암종의 진단횟수 비교 
        if pid_cancertypes.shape[0] > 0 :
            pid_ct_dict = {}
            # 하나 이상 일때 암종 (사전 key값에 입력) 하나씩 Nfreq를 구해서 사전 value값에 입력
            for pid_cancertype in pid_cancertypes :
#                 print(pid_condition[pid_condition['cancer_type']==\
#                                     pid_cancertype].sort_values(by='Nfreq')['Nfreq'].values[0])
                pid_ct_dict[pid_cancertype] = pid_condition[pid_condition['cancer_type']==\
                                                    pid_cancertype].sort_values(by='Nfreq')['Nfreq']\
                                                                  .values[0]
            #Nfreq을 횟수로 오름차순으로 정리 후 
            #print(pid_ct_dict)
            sorted_pid_ct_dict = {k: v for k, v in sorted(pid_ct_dict.items(),   
                                                          key=lambda item: item[1])}
            # 제일 큰 Nfreq가 마지막 element이므로 popitem()[0]를 하면 그 key값(=pid_cancertype)을 찾아 반환
            return sorted_pid_ct_dict.popitem()[0]
            
    # 아니라면 암종은 others임        
    else :
        return None

In [21]:
os.remove("patient_drug_numbers.txt")

In [22]:
def write_file(data):
    # 각 환자가 쓰는 화학항암제 종류를 아래 파일에 저장, 후에 
    # 한 환자가 쓰는 항암제 종류의 histogram을 작성하는데 사용
    file_name = "patient_drug_numbers.txt"
    with open(file_name, 'a') as x_file:
        x_file.write('{}\n'.format(data))

In [32]:
def plot_diagram(pid, xbgn, xter, pid_condition, pid_op, pid_patho,
                 pid_drug_chemo, pid_death, save_path, nrls) :

    mean_days_month = 365./12
    
    # 가로축의 범위를 결정
    xmin = -3
    xmax = ((xter-xbgn).days + 2)/mean_days_month
    
    # 저장되는 그림의 경로
    outname = os.path.join(save_path, str(pid).zfill(4))
    
    # Minortick 자동설정
    minorLocator   = AutoMinorLocator()
    #leftward_arrow = u'\u2193' 

    # 15가지 색깔 생성
    num_colors = 15
    cm = plt.get_cmap('gist_rainbow')
    colors = [cm(1.*i/num_colors) for i in range(num_colors)]
    
    # colors and markers for cyclic 
    custom_cycler = (cycler(color=colors) +
                     cycler(marker=Line2D.filled_markers))
    
    # specifying the kinds of drug names and dosages for this patient w/o duplicates.
    tmp_drug_chemo = pid_drug_chemo.loc[:, ('code', 'name', 'quantity', 
                                            'dose_unit_source_value')]\
                                   .drop_duplicates()
    # title = easy identifier of drugs and dosages
    tmp_drug_chemo['name'] = tmp_drug_chemo['name'].astype('category')
    tmp_drug_chemo.drop(columns = ['quantity', 'dose_unit_source_value'], 
                        inplace = True)
    # number of categorical variable for title
    tmp_drug_chemo['drug_factor'] = pd.factorize(list(tmp_drug_chemo['name'].values))[0]      
    write_file("{},{}".format(str(pid).zfill(4), 
                                      len(tmp_drug_chemo['name'].unique())))                                          
    # add title to column of pid_drug_chemo
    pid_drug_chemo = pd.merge(pid_drug_chemo, 
                              tmp_drug_chemo.drop(columns=['name']), 
                                                  on='code')
    # 항암제의 종류에 따라서 그림의 세로 길이를 바꿈
    if len(tmp_drug_chemo['name'].unique()) < 3 :
        fig, ax = plt.subplots(1, 1, figsize=(15, 2+ len(tmp_drug_chemo['name'].unique())))
    elif len(tmp_drug_chemo['name'].unique()) > 30 :
        fig, ax = plt.subplots(1, 1, figsize=(15, 30))
    else :
        fig, ax = plt.subplots(1, 1, figsize=(15, 2+len(tmp_drug_chemo['name'].unique())))
    fig.patch.set_facecolor('white')
    ax.set_prop_cycle(custom_cycler)
    # plotting drug information by drug_factor
    # 항암제 종류별로 색깔과 모양을 달리해 가며 그림에 점을 찍는다. 
    for factor, group in pid_drug_chemo.groupby('drug_factor'):
        ax.plot(group['lapse_months'], group['name'].astype('category'), 
                linestyle='None', alpha=0.7)

    # 암진단 받은 기록이 있는 경우
    if len(pid_condition) > 0 :
        ymin, ymax = ax.get_ylim()
        pid_condition.sort_values(by=['Nfreq'], 
                                  ascending=False, inplace=True)

        cnt = 1
        y_line = 0
        # 암진단 기록 하나씩 작업 
        pid_condition.sort_values(by='lapse_months', inplace=True)
        for lapse_month, condition_date, kcd, cancer_type in pid_condition[['lapse_months', 
                                                              'condition_start_date',
                                                              'condition_source_value', 
                                                              'cancer_type']].values:
            # 오렌지색 세로줄을 표시한다. 
            ax.axvline(lapse_month, color = 'tab:orange', linestyle='dotted', 
                       label="{}. {} ({}) at {} ({:5.1f})"\
                      .format(str(cnt), kcd, cancer_type, 
                              condition_date.strftime('%Y/%m/%d'), lapse_month))
            # 여러 개의 진단이 같은 날에 내려질 때 세로줄 오른쪽에 붙은 번호의 세로축을 미세하게 조정
            if cnt > 1 :
                # procedure event간의 간격이 매우 촘촘하다면
                if lapse_month - pre_lapse_month < 2 :
                    y_line += (ymax-ymin)/20
                    
                else :
                    y_line = 0
            # 번호를 오렌지색 세로줄 옆에 적어넣는다. 
            ax.text(lapse_month+(xmax-xmin)/100, y_line, str(cnt), 
                   horizontalalignment='center', fontsize=15,
                   verticalalignment='center', color='tab:orange')
            cnt += 1
            pre_lapse_month = lapse_month
        # 그림 제목을 작성    
        # 만약 원발성 암이 others라고 나왔을 경우 같은 날에 다른 암종을 진단받은 것이 있으면 그 암종이 원발성 암
        pid_cancertype = None
        pid_condition.sort_values(by=['Nfreq'], 
                                  ascending=False, inplace=True)
        # 만약 pid_condition의 cancer_type의 unique값이 1개가 아니고 원발암의 종류가 others일 때
        if len(pid_condition['cancer_type'].unique())!=1 and \
           pid_condition.loc[0, 'cancer_type'] == 'others' :
            pid_cancertype = set_representative_cancertype_in_case_of_others(pid_condition)
            if pid_cancertype != None:
                #print(pid_cancertype)
                #print(pid_condition)
                titlename = "pid = "+str(pid)+", The date of the first record is {}, Diag.: {} ({}) at {}"\
                        .format(xbgn.strftime('%Y/%m/%d'),pid_cancertype, 
                                pid_condition[pid_condition['cancer_type']==pid_cancertype]\
                                ['condition_source_value'].values[0],
                                pid_condition[pid_condition['cancer_type']==pid_cancertype]\
                                ['condition_start_date'].values[0].strftime('%Y/%m/%d'))
                
        if pid_cancertype == None :
            titlename = "pid = "+str(pid)+", The date of the first record is {}, Diag.: {} ({}) at {}"\
                        .format(xbgn.strftime('%Y/%m/%d'),
                               pid_condition.loc[0, 'cancer_type'], 
                               pid_condition.loc[0, 'condition_source_value'],
                               pid_condition.loc[0, 'condition_start_date'].strftime('%Y/%m/%d'))
            
    # 암진단 받은 기록이 존재하지 않는다면, 
    else :
        titlename = 'DUNNO, '+str(pid)+' 환자는 condition table에 KCD 값이 존재하지 않음' + \
                    'The date of the first record is {}'.format(xbgn.strftime('%Y/%m/%d'))
    
    # plotting death information 
    # 사망원인과 일시를 90도로 기울여서 그림에 작성
    ymin, ymax = ax.get_ylim()
    if len(pid_death) > 0:
        ax.text(pid_death['lapse_months'].astype('int'), (ymax-ymin)/100, 
                '{}\n c.d.: {}'.format(pid_death['death_date'].values[0].strftime('%Y/%m/%d'), 
                                     pid_death['cause_source_value'].values[0]),
               rotation=90, fontsize=13, color='tab:red')
        
    # plotting opeation information    
    if len(pid_op) > 0:
        cnt = 1
        y_line = 0
        # 수술을 하나씩 작업 
        for lapse_month, opname in pid_op[['lapse_months', '수술코드명']].values :
            ax.axvline(x=lapse_month, color = 'tab:blue', linestyle='dashed',
                       label="{}. {} at ({:5.1f})".format(str(cnt), opname, lapse_month))
            
            if cnt > 1 :
                if lapse_month - pre_lapse_month < 2 :
                    y_line += (ymax-ymin)/20
                else :
                    y_line = 0
            # 파란색 세로줄 오른쪽에 숫자 레이블을 작성 
            ax.text(lapse_month+(xmax-xmin)/100, y_line, str(cnt), horizontalalignment='center', 
                    verticalalignment='center', fontsize=15, color='tab:blue')
            cnt += 1
            pre_lapse_month = lapse_month
            
    # plotting patho information    
    if len(pid_patho) > 0:
        cnt = 1
        y_line = 0
        for lapse_month, opname in pid_patho[['lapse_months', '2.0 처방코드 영문명']].values :
            ax.axvline(x=lapse_month, color = 'tab:green', linestyle='dashdot', 
                       label="{}. {} at ({:5.1f})".format(str(cnt), opname, lapse_month))
            if lapse_month - pre_lapse_month < 2 :
                y_line += (ymax-ymin)/20
            else :
                y_line = 0
            ax.text(lapse_month+(xmax-xmin)/100, y_line, str(cnt), horizontalalignment='center', 
                    verticalalignment='center', fontsize=15, color='tab:green')
            cnt += 1
            pre_lapse_month = lapse_month
            
            
    #ax.legend(fontsize = 13, bbox_to_anchor=(1, -0.2), fancybox=True, shadow=True)
    # 범례를 그림 아래에
    ax.legend(fontsize = 13, bbox_to_anchor=(1, -0.2), fancybox=True, shadow=True)
    if isinstance(nrls, pd.DataFrame) :
        for i, rgmline in enumerate(sorted(nrls[nrls['rgmline']!=0]['rgmline'].unique())) :
            rgmline_span = nrls[nrls['rgmline']==rgmline]['lapse_months'].values
            # 화학항암제를 1번만 사용했다면 regimen을 표시하는 너무 작아서 음영이 안 나와서 조작함
            if len(rgmline_span) == 1 :
                ax.fill_between(np.arange(rgmline_span[0]-0.25, rgmline_span[0]+0.25, 0.1),
                                0, 1, where=None, alpha=0.2, facecolor=colors[i], 
                                transform=ax.get_xaxis_transform())
            # regimen 표시하는 음염이 충분한 면적으로 나옴
            else :
                ax.fill_between(rgmline_span, 0, 1, where=None, alpha=0.2, 
                                facecolor=colors[i], transform=ax.get_xaxis_transform())
        
        ax.text(0.5, 0.5, str(len(nrls[nrls['rgmline']!=0]['rgmline'].unique())),
                size=120, rotation=0, color='tab:gray', 
                ha="center", va="center", alpha=0.3, transform=ax.transAxes)
    else :
        ax.text(0.5, 0.5, str(nrls), size=120, rotation=0, color='tab:gray', 
                ha="center", va="center", alpha=0.3, transform=ax.transAxes)
    ax.set_xlim([xmin, xmax])
    ax.set_yticklabels(tmp_drug_chemo['name'].unique().to_numpy(), 
                       fontproperties=font_prop)    
    ax.set_title(titlename, fontsize=15, fontproperties=font_prop)
    ax.set_xlabel('Lapse months since cancer diagnosis', fontsize=14)
    ax.set_ylabel('Chemo drugs', fontsize=14)
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(color='gray', linestyle='--', alpha=0.5, linewidth = 0.5)
    ax.xaxis.set_minor_locator(minorLocator)
    ax.tick_params(which='major', length=7)
    ax.tick_params(which='minor', length=5)
    plt.savefig(outname, bbox_inches="tight")
    #plt.show()
    plt.close(fig)

regimen line 구분을 위한 일반적인 regimen line에서 항암제 투여하는 스케줄 특징

1. 약 이름을 통한 regimen 구분 regimen  
 1) 항암제 regimen에서는 같은 항암제를 계속 사용하는 것은 아니고 2개를 쓰다가 중간에 1개만 쓰기도 하고 새로운 항암제가 들어오기 전까지는 다시 2개를 쓰는 등 몇 개의 사례에서 항암제 이름으로 regimen을 구분할 때는 i-3 ~ i-1번째 행이 i번째 행의 부분집합이거나 그 역이 성립하면 즉, 
\begin{align}
\{name_{i-1}\} \supset \{name_{i}\}\\
\{name_{i-1}\} \subset \{name_{i}\}
\end{align}
 

In [24]:
def get_privous_drugs(drughx) :
    
    # get 3 previous drugs
    # input : drug histories df
    # output: drug histories df added new column, 'drug_pattern'
    # drug_pattern is union of name of drugs of 3 previous drug historie환
    # drug_pattern 칼럼은 직전 3회분 투여에 사용된 약제들의 합집합
    for i in range(0, len(drughx)):
        # 공집합을 선언, i=0일 때 drug_pattern은 공집합
        drug_pattern = set({})
        # drug_pattern 작성을 위해 j>=i-3보다 크거나 같으면서 j>=0인 동안 while문을 도는데
        # i=1일 때는 while 1회 반복, i=2일 때는 while 2회반복, i=3일 때는 while 3회 반복
        j = i-1
        while j >= i-3 and j >= 0:
            # 합집합 계산
            drug_pattern = drug_pattern.union(drughx.loc[j, 'name'])
            j -= 1
        # 집합을 문자열로 변환하여 저장
        drughx.loc[i, 'drug_pattern'] = ','.join([x for x in drug_pattern])
    # 다시 문자열을 집합으로 변환    
    drughx.loc[:,'drug_pattern'] = drughx['drug_pattern'].apply(lambda x: set(x.split(",")))
    # drug_pattern이 추가된 drughx df 반환
    return drughx
    

In [25]:
def regimen_by_subset_drugs(drughx) :
    
    # 시간 순으로 두었을 때 i번째 항암제가 i-3 ~ i-1번째, 3회분 투여한 항암제들의 합집합의 부분집합이라면 
    # regimen line이 유지되고 있다고 판단하는 알고리즘
    # 부분집합 여부를 판단해서 그렇다면 같은 regimen으로 판단하여 rgmline을 유지, 부분집합이 아니라면, 
    # 다른 regimen으로 판단하여 i-1번째 rgmline 값에 1을 더해 i번째 rgmline으로 넣는다. 
    
    # 약제 투약 정보가 시작되면 일단 regimen line은 1이 된다. 
    drughx['rgmline'] = int(1)
    for i in range(1, len(drughx)):
        # 부분집합인가 ?
        if drughx.loc[i, 'name'].issubset(drughx.loc[i, 'drug_pattern']) :
            # rgmline 유지
            drughx.loc[i, 'rgmline'] = drughx.loc[i-1, 'rgmline']
        # 아닌가?
        else :
            # i-1번째 rgmline에서 1을 더해 i번째 rgmline에 입력
            drughx.loc[i, 'rgmline'] = drughx.loc[i-1, 'rgmline'] + int(1)
        #print(drughx.iloc[i])
    drughx['rgmline']
    return drughx

def adjuvant_regimen_by_drugname(drughx) :
    
    # cisplatin이나 mitomycin_c를 단독으로 사용하는 regimen이 있다면
    # 그 regimen을 0으로 만들어주는 함수
    adjuvant_drugs = set(['cisplatin', 'mitomycin_c'])
    # 1이상의 regimen을 순차적으로 돌리면서 각 regimen이 adjuvant인지 확인한다. 
    for i in range(1, drughx['rgmline'].unique().max()+1) :
        # i번쨰 regimen의 약제의 가지수를 계산
        drugname_this_regimen = np.unique(drughx[drughx['rgmline']==i]['name'].values)
        # 약제 가지수가 1개나 2개라면 
        if drugname_this_regimen.shape[0]==1 or \
          drugname_this_regimen.shape[0]==2:
            # 만약 i번째 regimen의 모든 약제가 adjuvant_drugs의 부분집합이라면, 
            # regimen을 0으로 변경하고 하나라도 부분집합이 아니라면, regimen 유지
            all_inclusion = True
            for j in drugname_this_regimen :
                if not j.issubset(adjuvant_drugs) :
                    # adjuvant_drugs에 속하지 않는 투약 회차 발견, regimen유지
                    all_inclusion = False
                    break
            if all_inclusion :
                # i번째 regimen의 모든 약제가 adjuvant_drugs의 부분집합인 경우
                drughx.loc[drughx['rgmline']==i, 'rgmline'] = 0
    return drughx

In [45]:
def patho_cancel(rgm_hx) :
    
    adjuvant_limit = 7 * 8
    # 병리검사 항목이 존재하는 날들을 추출
    patho_dates = np.unique(rgm_hx[rgm_hx['patho'].notnull()]['date'].values)
    
    for patho_date in patho_dates :
        if rgm_hx.iloc[np.where((rgm_hx['date']==patho_date) & \
                                (rgm_hx['patho'].notnull()))].shape[0] > 1 :
            print(rgm_hx.iloc[np.where((rgm_hx['date']==patho_date) & \
                                (rgm_hx['patho'].notnull()))])
        patho_index = rgm_hx.iloc[np.where((rgm_hx['date']==patho_date)& \
                                   (rgm_hx['patho'].notnull()))].index.max()
        
        # 병리검사일이 1번째 약제투여일 이전이라면 그 약제 투여일과의 시간차이 adjuvant_limit 이하라면,
        # index=1부터 끝까지 모두 regimen line들을 1씩 감소
        if patho_index == 0 :
            # adjuvant case
            if (rgm_hx.loc[patho_index+1, 'date'] - \
                patho_date) / np.timedelta64(1, 'D') < adjuvant_limit :
                rgm_hx.loc[:,'rgmline'] = rgm_hx['rgmline'].apply(lambda x: x-1)
        # 병리검사일이 마지막 약제투여일 다음이고 병리검사 이전 약제투여 regimen이 1일 경우엔
        # regimen을 0으로 만들어준다. neoadjuvant case
        elif patho_index == rgm_hx.shape[0]-1 :
            if rgm_hx.loc[patho_index-1, 'base_rgmline'] == 1 :
                rgm_hx.loc[:, 'rgmline'] = 0

        else :
            # 1)병리검사일이 약제 투여일 사이 어디고 병리검사일 바로 직전 행과 바로 다음 행의 regimen line이 차이있다면, 
            if rgm_hx.loc[patho_index-1, 'rgmline']!=\
              rgm_hx.loc[patho_index+1, 'rgmline'] :
                # 2)병리검사 이전 약제투여의 regimen이 1일 경우 이를 0으로 만들어준다. neoadjuvant case
                if rgm_hx.loc[patho_index-1, 'base_rgmline'] == 1 :
                    rgm_hx.loc[:patho_index, 'rgmline'] = 0

                # 3) 병리검사일 다음 약제투여 시작일이 병리검사일과의 시간차이가 adjuvant_limit 이하며,
                #   병리검사 다음 행의 regmien line을 모두 1씩 감소, # adjuvant case
                if (rgm_hx.loc[patho_index+1, 'date'] - \
                    patho_date) / np.timedelta64(1, 'D') < adjuvant_limit :
                    rgm_cancel = rgm_hx.loc[patho_index+1,'base_rgmline']
                    rgm_hx.loc[patho_index+1:,'rgmline'] = \
                        rgm_hx.loc[patho_index+1:,'rgmline'].apply(lambda x: x-1)

    return rgm_hx

In [27]:
def weekly_regimenline(pid_drug_chemo, pid_op, pid_patho) :
    
    # 1.약제가 없거나 혹은 cisplatin, mitomycin_c 중 하나만 가진 경우 regimen = 0
    drug_names = pid_drug_chemo.loc[:, 'name'].drop_duplicates().values
    # drug_names이 비어 있거나 한 개만 있는데 cisplatin, mitomycin_c 일 경우엔 0를 반환
    if drug_names.shape[0] == 0 :
        return 0
    if len(drug_names) == 1 :
        if re.match('cisplatin|mitomycin_c', drug_names[0]) :
            return 0
    
    # 2. 약제 이름으로 regimen을 추정하는데 
          
    #    현재 시점을 기준으로 지난 3번의 투여 약제 정보를 저장하는 칼럼(=drug_pattern)을 생성
    #    1) 직전 3번의 투여 기록에서 나오지 않은 약제는 새로운 약제로 보고 regimen을 1 증가시킴
    #    2) 이 경우는 같은 regimen이지만 새로운 약이 1-2회 늦게 등장하는 경우는 필요 이상으로 잘게
    #      regimen을 쪼개는 현상이 있을 수 있음
    drug_hx = pid_drug_chemo[['drug_exposure_start_date', 'name', 'lapse_months']]\
                         .rename(columns = {'drug_exposure_start_date':'date'})
    # date.dt를 Timestamp로 변경
    drug_hx.loc[:, 'date'] = pd.to_datetime(drug_hx['date'])
    # weekly로 resampling하며 date index label은 주 시작일이 됨 약제는 모두 join 시켜 string으로
    drug_weekly_hx = drug_hx.set_index('date').resample('W', label='left')['name']\
                        .apply(','.join)
    # date index label을 3일 이후로 옮겨주어 regimen block shift 현상 최소화함
    drug_weekly_hx.index = drug_weekly_hx.index + pd.DateOffset(days=3)
    drug_weekly_hx = drug_weekly_hx.reset_index()
    # 약제 이름을 중복 제거하여 다시 join 시켜 string으로 
    drug_weekly_hx.loc[:, 'name'] = drug_weekly_hx['name']\
                                        .apply(lambda x: ','.join([i for i in set(x.split(","))]))
    # 약제 투여 기록이 없는 주는 null값을 입력해주어
    drug_weekly_hx.loc[:, 'name'] = drug_weekly_hx['name']\
                                 .apply(lambda x: np.nan if len(x)==0 else x)
    # null row 제거
    drug_weekly_hx.dropna(subset=['name'], inplace=True)
    # 남은 투약기록들의 index를 날짜 오름차순으로 1씩 순차적으로 재순서화
    drug_weekly_hx.reset_index(inplace=True)
    # 기존의 index는 제거
    drug_weekly_hx.drop(columns=['index'], inplace=True)
    # 약제들의 이름을 집합으로 변경
    drug_weekly_hx.loc[:,'name'] = drug_weekly_hx['name'].apply(lambda x: set(x.split(",")))
    # 이전 3번의 투여한 기록이 있는 약제를 담는 칼럼 (drug_pattern) 생성
    drug_weekly_hx = get_privous_drugs(drug_weekly_hx)
    # 이번에 나온 약제가 drug_pattern의 부분집합이라면 regimen은 유지 아니면, regimen 1 증가
    drug_rgm_weekly_hx = regimen_by_subset_drugs(drug_weekly_hx)
    # 특정 regimen이 cisplatin, mitomycin_c 단독으로 쓰인 것이 있다면 그 regimen은 0으로 변경
    drug_rgm_weekly_hx = adjuvant_regimen_by_drugname(drug_rgm_weekly_hx)
    
    # patho 정보와 결합하기 위한 칼럼 정보 변경
    rgm_weekly_hx = drug_rgm_weekly_hx[['date', 'name', 'rgmline']]\
                      .rename(columns={'name':'drugs'})
    rgm_weekly_hx.loc[:, 'drugs'] = rgm_weekly_hx['drugs']\
                                 .apply(lambda x: ','.join(x))
    rgm_weekly_hx['patho'] = np.nan

    # patho 정보가 1개라도 있을 경우 
    if pid_patho.shape[0] > 0 :
        # patho 정보를 regimen df와 결합하기 위한 전처리
        pid_patho['procedure_date'] = pd.to_datetime(pid_patho['procedure_date'])
        pathos = pid_patho['procedure_date'].to_frame(name='date')
        pathos.loc[:, 'date'] = pd.to_datetime(pathos['date'])

        rgm_weekly_hx_columns = rgm_weekly_hx.columns
        rgm_weekly_hx_columns = rgm_weekly_hx_columns[rgm_weekly_hx_columns!='date']
        for rgm_weekly_hx_column in rgm_weekly_hx_columns :
            pathos[rgm_weekly_hx_column] = np.nan
        pathos['patho'] = pid_patho['2.0 처방코드 영문명'].values  
        pathos['patho'] = pathos['patho'].str.replace(' ', '_')
        pathos = pathos.groupby('date')['patho'].apply(','.join).reset_index()
        # regimen 정보와 patho 정보 결합
        rgm_weekly_hx = pd.concat([rgm_weekly_hx, pathos], 
                                  axis=0, ignore_index=True).sort_values(by='date')
        rgm_weekly_hx = rgm_weekly_hx.reset_index().drop(columns=['index'])
        rgm_weekly_hx['rgmline'].fillna(0, inplace=True)
        rgm_weekly_hx['base_rgmline'] = rgm_weekly_hx['rgmline']
        # 3. patho 정보와 drug 정보 사이의 기간 등을 이용한 regimen 재조정
        #   1) patho 앞 뒤에서 regimen이 변경되지 않는다면 patho 정보 무시 함수를 나오고, 
        #    regimen이 바뀐다면 중요한 patho로 보고 아래 절차를 수행
        #   2) patho 한 후 8주 이내에 시작한 투약 regimen이 존재한다면, 이는 adjuvant로 regimen을 0으로 변경
        #   3) patho 직전에 regimen=1인 정보가 있다면 기간에 상관없이 patho 일자 이전 regimen은 0으로 변경
        #   4) 만약 patho 앞쪽 regimen이 2개 이상 있다면 앞쪽 regimen은 patho로 인한 값의 변경없음
        rgm_hx = patho_cancel(rgm_weekly_hx)
    else :
        rgm_hx = rgm_weekly_hx.copy()
    # 4. regimen에서 사용 약제를 이름을 합집합하여, sorting한 후에 
    #   regimen간의 Levenshtein distance로 검증할 수 있음 (future work)
    rgm_hx['rgmline'] = rgm_hx['rgmline'].apply(lambda x: 0 if x<0 else x)
    rgm_hx.loc[:, 'date'] = rgm_hx['date'].dt.date 
    return rgm_hx

In [44]:
save_path_head = 'regimen_diagrams/chemo'
os.makedirs(save_path_head, exist_ok=True)
personid_cancertypes_df = pd.DataFrame(columns = ['person_id', 'cancer_type', 'KCD'])
mean_days_month = 365./12
regimen_df = pd.DataFrame(columns=['person_id', 'regimen_line'])
#pid_chemo = [1363]
for i, pid in enumerate(tqdm(pid_chemo)) :
    #print(pid)
    input_personid_cancertypes = [pid]
    # find the first and the last of all medical records for a patient.
    # 암진단(condition_pd), 화학항암제(drug_df), 선택된 키워드를 포함하는 수술(procedure_df)만 포함
    ts = pd.concat([condition_pd[(condition_pd['person_id'] == pid)& 
                                 (condition_pd['condition_source_value']\
                                  .str.contains('C', regex=False))].loc[:, 'condition_start_date'],
                  drug_df[drug_df['person_id']==pid].loc[:, 'drug_exposure_start_date'],
                  procedure_df[procedure_df['person_id'] == pid].loc[:, 'procedure_date'],
                  patho_df[patho_df['person_id']==pid].loc[:, 'procedure_date'], 
                  death_pd[death_pd['person_id'] == pid].loc[:, 'death_date']], 
                  ignore_index=True)
    
    # xbgn is the first record date, xter is the last record date
    xbgn, xter = ts.min(), ts.max()
    # patient diagnosis information 
    # patient id and ICD 10 code start with 'C'
    pid_condition = condition_pd[(condition_pd['person_id'] == pid) & 
                                 (condition_pd['condition_source_value'].str.contains('C', regex=False))]\
                                    .loc[:, ('condition_start_date', 'condition_source_value')]
    # keep only first three chracters of condition_sourcec_value
    pid_condition.loc[:, 'condition_source_value'] = pid_condition['condition_source_value'].apply(
                                                                   lambda x: x[:3])
    pid_condition.sort_values(by='condition_start_date', ascending=True, inplace=True)
    cancer_freq_df = pid_condition['condition_source_value']\
                                  .value_counts().to_frame().reset_index()\
                                  .rename(columns = {'index':'condition_source_value', 
                                                     'condition_source_value':'Nfreq'})
    #print(pid_condition['condition_source_value'].value_counts())
    # keep only the first diagnoses
    pid_condition.drop_duplicates(subset='condition_source_value', keep='first', inplace = True)
    #pid_condition.sort_values(by='condition_source_value', inplace=True)
    pid_condition['cancer_type'] = pid_condition['condition_source_value']\
                                       .apply(lambda x: pd.Series(cancer_type[cancer_type['enum']\
                                                                 .str.contains(x)]['cancer']\
                                                                 .values.ravel()))
    pid_condition.dropna(subset=['cancer_type'], inplace=True)
    pid_condition = pd.merge(pid_condition, cancer_freq_df, on='condition_source_value', how='left')
    pid_condition['lapse_months'] = pid_condition['condition_start_date'].apply(
                                                  lambda x: (x - xbgn).days/mean_days_month)
    try :
        pid_cancertype = pid_condition.sort_values(by='Nfreq', 
                                                   ascending=False).loc[0, 'cancer_type']
    except :
        pid_cancertype = None
        
    # 만약 pid_condition의 cancer_type의 unique값이 1개가 아니고 원발암의 종류가 others일 때
    if len(pid_condition['cancer_type'].unique())!=1 and pid_cancertype == 'others' :
        pid_cancertype = set_representative_cancertype_in_case_of_others(pid_condition)
        if pid_cancertype == 0 :
            pid_cancertype = 'others'
    
    # 환자가 진단 받은 주 암종에 따라서 그림을 저장
    if pid_cancertype :        
        subdirname = pid_cancertype
        input_personid_cancertypes.append(pid_cancertype)
        input_personid_cancertypes.append(pid_condition[pid_condition['cancer_type']== pid_cancertype]\
                                                      ['condition_source_value'].values[0])
    else :
        # KCD 코드가 없는 환자일 경우 그림을 저장할 곳을 지정 
        print(pid)
        subdirname = 'DUNNO'
        input_personid_cancertypes.append(subdirname)
        input_personid_cancertypes.append("")
     
    #print(input_personid_cancertypes)    
    personid_cancertypes_df.loc[len(personid_cancertypes_df)] = input_personid_cancertypes
    
    save_path = os.path.join(save_path_head, subdirname)
    os.makedirs(save_path, exist_ok=True)
   
    # chemo drug information
    pid_drug_chemo_tmp = drug_df[drug_df['person_id']==pid]\
                                .loc[:, ('drug_exposure_start_date', 'drug_exposure_end_date',
                                        'stop_reason', 'drug_source_value', 'refills', 
                                        'quantity', 'days_supply', 'sig',
                                        'dose_unit_source_value', 'code', 'name')]
    # 화학항암제만 남기고 나머지는 없앤다. 
    pid_drug_chemo = pid_drug_chemo_tmp[pid_drug_chemo_tmp['drug_source_value']\
                                       .isin(chemo_drug['code'])]
    pid_drug_chemo['lapse_months'] = pid_drug_chemo['drug_exposure_start_date'].apply(
                                        lambda x: (x - xbgn).days/mean_days_month)
    # 공백문자 대신에 줄바꿈문자를 넣는 이유는 yticklabel 너무 길 경우 그림이 그려지는 영역이 좁아져 그림의 설명력이 낮아짐
    #pid_drug_chemo['name'] = pid_drug_chemo['name'].str.replace(' ', '\n')
    
    # patient operation information 
    pid_op_tmp = procedure_df[procedure_df['person_id'] == pid]\
                             .loc[:, ('procedure_concept_id', 'procedure_date',
                                      '수술코드', '수술코드명')]
    # 키워드에 포함된 수술만 남기고 나머지는 전부 제거
    pid_op = pid_op_tmp[~pid_op_tmp['수술코드'].isnull()]
    pid_op['lapse_months'] = pid_op['procedure_date'].apply(lambda x: (x - xbgn).days/mean_days_month)
    # concept_id와 lapse_month가 동일할 경우 하나만 남기고 나머지 제거 
    pid_op.drop_duplicates(subset=['procedure_concept_id', 'lapse_months'], 
                           keep='last', inplace = True)
    pid_op.sort_values(by='lapse_months', inplace=True)
    
    # patient patho informatioin 
    pid_patho_tmp = patho_df[patho_df['person_id'] == pid]\
                                      .loc[:, ('procedure_concept_id', 'procedure_date',
                                               '2.0 처방코드', '2.0 처방코드 영문명')]
    pid_patho = pid_patho_tmp[~pid_patho_tmp['2.0 처방코드'].isnull()]
    pid_patho['lapse_months'] = pid_patho['procedure_date'].apply(lambda x: (x - xbgn).days/mean_days_month)
    # concept_id와 lapse_month가 동일할 경우 하나만 남기고 나머지 제거 
    pid_patho.drop_duplicates(subset=['procedure_concept_id', 'lapse_months'], 
                               keep='last', inplace = True)
    pid_patho.sort_values(by='lapse_months', inplace=True)
    # patient death information
    pid_death = death_pd[death_pd['person_id'] == pid]\
                        .loc[:, ('death_date', 'cause_source_value', 
                                 'death_type_concept_id')]
    pid_death['lapse_months'] = pid_death['death_date'].apply(lambda x: (x - xbgn).days/mean_days_month)
    # number of regimen lines
    #if len(pid_patho) > 0 :
    nrls = weekly_regimenline(pid_drug_chemo, pid_op, pid_patho)
    #nrls = total_regimenline1(pid_drug_chemo, pid_op, pid_patho)
    if isinstance(nrls, pd.DataFrame) :
        nrls['lapse_months'] = nrls['date'].apply(lambda x: (x - xbgn).days/mean_days_month)
        regimen_df.loc[len(regimen_df)] = [pid, len(nrls[nrls['rgmline']!=0]['rgmline'].unique())]
    else :
        regimen_df.loc[len(regimen_df)] = [pid, nrls]
    plot_diagram(pid, xbgn, xter, pid_condition, pid_op, pid_patho,
                 pid_drug_chemo, pid_death, save_path, nrls)

 12%|█▏        | 413/3534 [04:08<28:44,  1.81it/s]  

1703


 14%|█▍        | 495/3534 [04:56<28:55,  1.75it/s]

1378


 14%|█▍        | 497/3534 [04:57<29:48,  1.70it/s]

...

 97%|█████████▋| 3441/3534 [33:59<00:49,  1.90it/s]

5260


 99%|█████████▉| 3491/3534 [34:26<00:21,  2.01it/s]

808


100%|██████████| 3534/3534 [34:49<00:00,  1.69it/s]


In [ ]:
regimen_df[regimen_df['regimen_line']==0]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12.9, 5))
regimen_df['regimen_line'].hist(bins=12, ax=ax, grid=True, color='tab:gray', align='left')
N3 = regimen_df[regimen_df['regimen_line']>2]
N3['regimen_line'].hist(bins=9, ax=ax, grid=True, color='tab:orange', align='left',
                        label='N(R>2)='+str(N3.shape[0]), alpha=0.7)
ax.set_xlabel('Regimen lines', fontsize=16)
ax.set_ylabel('N', fontsize=16)
ax.tick_params(axis='both', labelsize=15)
ax.set_yscale('log')
ax.axvline(x=3, color='tab:blue', linewidth=2, 
           linestyle='dashed', label='N=3')
ax.legend(fontsize=14)

In [ ]:
regimen_df[regimen_df['regimen_line']==0].to_csv("rgm0.csv", index=False)

In [ ]:
# def get_ms_drugs(drughx) :
#     # get moving summation of drugs within +- 10days
#     for i in range(0, len(drughx)):
#         ms = set({})
#         ith_row_date = drughx.loc[i, 'date']
#         for j in range(0, len(drughx)) :
#             if j!= i :
#                 dgap = (ith_row_date-drughx.loc[j, 'date']) / np.timedelta64(1, 'D')
#                 if dgap >-10 and dgap < 10 :
#                     ms = ms.union(drughx.loc[j, 'name'])
                
#         drughx.loc[i, 'ms'] = ','.join([x for x in ms])
        
#     drughx.loc[:,'ms'] = drughx['ms'].apply(lambda x: set(x.split(",")))
#     return drughx

In [ ]:
# 부분집합을 이용한 daily과 weekly regimen line 추정

# def total_regimenline1(pid_drug_chemo, pid_op, pid_patho) :
    
#     # criterion for considering regimen as an adjuvant after or before operation 
#     #adjuvant_limit = 7 * 8
#     # After ordinary_regimen_period since the regimen has begun, 
#     # when a new drug is appeared, new regimen just began
#     #ordinary_regimen_period = 12*7
    
#     # 1.약제가 없거나 혹은 cisplatin, mitomycin_c 중 하나만 가진 경우 regimen = 0
#     drug_names = pid_drug_chemo.loc[:, 'name'].drop_duplicates().values
#     # drug_names이 비어 있거나 한 개만 있는데 cisplatin, mitomycin_c 일 경우엔 0를 반환
#     if drug_names.shape[0] == 0 :
#         return 0
#     if len(drug_names) == 1 :
#         if re.match('cisplatin|mitomycin_c', drug_names[0]) :
#             return 0
    
#     # 2. 약제 이름을 regimen을 추정하는데 
#     #   1) 오늘을 기준으로 9일 전 후까지의 약제를 저장하는 moving summation(=ms) 칼럼을 생성
#     #      --> 여러 약제가 하루만에 투여되기도 하지만 A(0일차)->B(1일차)->C(2일차)->7일후 
#     #          A(0)->B(1)-C(2) 패턴이 반복되는 것이 있으므로 앞뒤 9일 총 18일간은 맞은 약제는 
#     #          시점이 동일하다고 간주하겠다는 의미               
#     #   2) 오늘 기준으로 지난 90일동안 맞은 약제를 중복없이 모두 저장하는 칼럼(=drug_pattern)을 생성
#     #      ---> 같은 약을 일정한 간격을 주기로 (9일보다는 훨씬 긴 주기) 맞는 경우는 1)번 알고리즘만으로
#     #           해결할 수 없음 따라서 이런 경우 동일한 약제가 과거 90일안에 나왔던 경우라면 regimen이
#     #           유지되는 것으로 추정
#     drug_hx = pid_drug_chemo[['drug_exposure_start_date', 'name', 'lapse_months']]\
#                          .rename(columns = {'drug_exposure_start_date':'date'})
#     # date.dt를 Timestamp로 변경
#     drug_hx.loc[:, 'date'] = pd.to_datetime(drug_hx['date'])
#     # free hyper-parameters 결정
#     drug_hx.sort_values(by='date', inplace = True)
#     # 항암제 용량, 투여방법을 제거하면 같은 날에 같은 약이 들어간 행이 있을 수 있는데 제거 
#     drug_hx.drop_duplicates(keep = 'first', inplace=True)
#     # 항암제 투여한 날짜에 맞은 모든 항암제 이름을 모아서 집합(set)으로 변경하여 name칼럼에 저장
#     drug_hx = drug_hx.groupby('date')['name'].apply(set).reset_index()
 
#     if len(drug_hx) > 0 :
#         # drug를 현재 기준으로 중복없이 앞뒤 열흘치를 모두 합집합하여 ms라는 칼럼을 생성하여 값 입력 
#         #drug_hx = get_ms_drugs(drug_hx)
#         # drug를 현재 기준으로 중복없이 직전 90일치의 약제를 중복없이 저장하는 drug_pattern 칼럼 생성 후 값 입력
#         #print(drug_hx)
#         drug_hx = get_privous_drugs(drug_hx)
#         # 부분집합을 이용하여 regimen결정
#         drug_rgm_hx = regimen_by_subset_drugs(drug_hx)
#         # regimen 중 cisplatin과 mitomycin_c 인 것은 원래대로 돌린다. 
#         drug_rgm_hx = adjuvant_regimen_by_drugname(drug_rgm_hx)
        
#         # rgm_hx: 투약정보만 이용한 regimen 추정
#         rgm_hx = drug_rgm_hx[['date', 'name', 'rgmline']]\
#                      .rename(columns={'name':'drugs'})
#         rgm_hx.loc[:, 'drugs'] = rgm_hx['drugs']\
#                                      .apply(lambda x: ','.join(x))
#         rgm_hx['patho'] = np.nan
#         #rgm_hx.loc[:, 'rgmline']  = rgm_hx['rgmline'].apply(lambda x: x+1)
#         #print(rgm_hx)
#         #rgm_hx.loc[0, 'rgmline'] = rgm_hx.loc[1, 'rgmline']
#         #print(rgm_hx)
#         # regimen 시작일 저장
#         #regimen_start_dates = sorted(rgm_hx[rgm_hx['subsetTrue']==False]['date'].tolist())
        
        
        
#     # patho preprocessing
#     if pid_patho.shape[0] > 0 :
#         pid_patho['procedure_date'] = pd.to_datetime(pid_patho['procedure_date'])
#         pathos = pid_patho['procedure_date'].to_frame(name='date')
#         pathos.loc[:, 'date'] = pd.to_datetime(pathos['date'])
        
#         rgm_hx_columns = rgm_hx.columns
#         rgm_hx_columns = rgm_hx_columns[rgm_hx_columns!='date']
#         for rgm_hx_column in rgm_hx_columns :
#             pathos[rgm_hx_column] = np.nan
#         pathos['patho'] = pid_patho['2.0 처방코드 영문명'].values  
#         pathos['patho'] = pathos['patho'].str.replace(' ', '_')
#         pathos = pathos.groupby('date')['patho'].apply(','.join).reset_index()
#         rgm_hx = pd.concat([rgm_hx, pathos], axis=0, ignore_index=True).sort_values(by='date')
#         rgm_hx = rgm_hx.reset_index().drop(columns=['index'])
#         rgm_hx['rgmline'].fillna(0, inplace=True)
#         rgm_hx['base_rgmline'] = rgm_hx['rgmline']
        
#         rgm_hx = patho_cancel(rgm_hx)
#         #print("no resampling after patho cancel rgm_hx")
#         #print(rgm_hx)
    
#     if drug_names.shape[0] < rgm_hx['rgmline'].max() :
#         print("================ Weekly ================")
#         rgm_hx = rgm_hx[rgm_hx['drugs'].notnull()]
#         drug_weekly_hx = rgm_hx.set_index('date').resample('W')['drugs'].apply(','.join).reset_index()
        
#         drug_weekly_hx['drugs'] = drug_weekly_hx['drugs'].apply(lambda x: np.nan if len(x)==0 else x)
#         drug_weekly_hx.rename(columns={'drugs':'name'}, inplace=True)
#         drug_weekly_hx.dropna(subset=['name'], inplace=True)
#         drug_weekly_hx.reset_index(inplace=True)
#         drug_weekly_hx.drop(columns=['index'], inplace=True)
#         drug_weekly_hx.loc[:,'name'] = drug_weekly_hx['name'].apply(lambda x: set(x.split(",")))
#         drug_weekly_hx = get_privous_drugs(drug_weekly_hx)
#         drug_rgm_weekly_hx = regimen_by_subset_drugs(drug_weekly_hx)
#         drug_rgm_weekly_hx = adjuvant_regimen_by_drugname(drug_rgm_weekly_hx)
        
#         # rgm_hx: 투약정보만 이용한 regimen 추정
#         rgm_weekly_hx = drug_rgm_weekly_hx[['date', 'name', 'rgmline']]\
#                           .rename(columns={'name':'drugs'})
#         rgm_weekly_hx.loc[:, 'drugs'] = rgm_weekly_hx['drugs']\
#                                      .apply(lambda x: ','.join(x))
#         rgm_weekly_hx['patho'] = np.nan
#         rgm_weekly_hx.loc[:, 'rgmline']  = rgm_weekly_hx['rgmline'].apply(lambda x: x+1)
#         #rgm_weekly_hx.loc[0, 'rgmline'] = 1
#         #rgm_weekly_hx.loc[0, 'rgmline'] = rgm_weekly_hx.loc[1, 'rgmline']
#         #print(rgm_weekly_hx)
#         if pid_patho.shape[0] > 0 :
#             pid_patho['procedure_date'] = pd.to_datetime(pid_patho['procedure_date'])
#             pathos = pid_patho['procedure_date'].to_frame(name='date')
#             pathos.loc[:, 'date'] = pd.to_datetime(pathos['date'])

#             rgm_hx_columns = rgm_hx.columns
#             rgm_hx_columns = rgm_hx_columns[rgm_hx_columns!='date']
#             for rgm_hx_column in rgm_hx_columns :
#                 pathos[rgm_hx_column] = np.nan
#             pathos['patho'] = pid_patho['2.0 처방코드 영문명'].values  
#             pathos['patho'] = pathos['patho'].str.replace(' ', '_')
#             pathos = pathos.groupby('date')['patho'].apply(','.join).reset_index()
#             rgm_weekly_hx = pd.concat([rgm_weekly_hx, pathos], axis=0, ignore_index=True).sort_values(by='date')
#             rgm_weekly_hx = rgm_weekly_hx.reset_index().drop(columns=['index'])
#             rgm_weekly_hx['rgmline'].fillna(0, inplace=True)
#             rgm_weekly_hx['base_rgmline'] = rgm_weekly_hx['rgmline']
#             rgm_hx = patho_cancel(rgm_weekly_hx)
#         else :
#             rgm_hx = rgm_weekly_hx.copy()
#         #print("weekly resampling after patho cancel rgm_hx")
#         #print(rgm_hx)
#     rgm_hx['rgmline'] = rgm_hx['rgmline'].apply(lambda x: 0 if x<0 else x)
#     rgm_hx.loc[:, 'date'] = rgm_hx['date'].dt.date 
#     #print(rgm_hx)
#     return rgm_hx

In [ ]:
# def total_regimenline(pid_drug_chemo, pid_op, pid_patho) :
    
#     # criterion for considering regimen as an adjuvant after or before operation 
#     adjuvant_limit = 7 * 8
#     # After ordinary_regimen_period since the regimen has begun, 
#     # when a new drug is appeared, new regimen just began
#     ordinary_regimen_period = 12*7
    
# #     # 용량을 나타난 정보를 제거 
# #     pid_drug_chemo.loc[:, 'name'] = pid_drug_chemo['name'].str.replace(r'\[[0-9.]+m[gl]\]','')
# #     # tab/inj 등 투여 방법을 나타내는 정보 제거
# #     pid_drug_chemo.loc[:, 'name'] = pid_drug_chemo['name']\a
# #                                         .apply(lambda x: re.sub(r'\ntab|\ninj', '', x.strip('\n')))
# #     # 줄바꿈 문자를 underbar로 변경
# #     pid_drug_chemo.loc[:, 'name'] = pid_drug_chemo['name'].str.replace('\\n','_')
    
#     # 1. regimen 간의 시간 차의 최소값 예상
#     # 2. 같은항암제가 다른 regimen에 속할 시간 차 예상
#     # 항암제 이름으로 regimen을 구분하는 필요한 칼럼만 저장하고 긴 날짜이름을 짧게 변경
#     drug_hx = pid_drug_chemo[['drug_exposure_start_date', 'name', 'lapse_months']]\
#                          .rename(columns = {'drug_exposure_start_date':'date'})
#     # date.dt를 Timestamp로 변경
#     drug_hx.loc[:, 'date'] = pd.to_datetime(drug_hx['date'])
#     # free hyper-parameters 결정
#     drug_hx.sort_values(by='date', inplace = True)
#     # 항암제 용량, 투여방법을 제거하면 같은 날에 같은 약이 들어간 행이 있을 수 있는데 제거 
#     drug_hx.drop_duplicates(keep = 'first', inplace=True)
#     # 항암제 투여한 날짜에 맞은 모든 항암제 이름을 모아서 집합(set)으로 변경하여 name칼럼에 저장
#     drug_hx = drug_hx.groupby('date')['name'].apply(set).reset_index()
 
#     # 투약을 시작한 날짜로부터 경과일을 integer type으로 제작
#     #drug_hx['lapsed_days'] = ((drug_hx['date']-drug_hx['date'].iloc[0]) \
#     #                          / np.timedelta64(1, 'D')).astype(int)
    
#     if len(drug_hx) > 1 :
        
#         drug_hx = get_ms_drugs(drug_hx)
#         drug_rgm_hx = regimen_by_subset_drugs(drug_hx) 
#         # 부분집합만으로 충분하지 않아서 이전에 맞지 않았던 새로운 항암제를 투여받고 현재 regime이 유지 된지 12주가 
#         # 넘은 시점이라면 regimen line이 바뀌었다고 판단한다. 
# #         current_regimen_line = drug_hx.loc[0, 'rgmline']
# #         drugnames_shown = drug_hx.loc[0, 'name']
# #         for i in range(1, len(drug_hx)):
# #             bgn_current_regimen_line_date = drug_hx[drug_hx['rgmline']==\
# #                                                     current_regimen_line]['date'].min()
# #             lapsed_days_current_regimen = (drug_hx.loc[i, 'date'] - bgn_current_regimen_line_date)\
# #                                           / np.timedelta64(1, 'D')
# #             current_drugs = drug_hx.loc[i, 'name']
# #             for current_drug in current_drugs :
# #                 if not current_drug in drugnames_shown and \
# #                   lapsed_days_current_regimen > ordinary_regimen_period :
# #                     # 두 조건을 만족하면 regimen이 1 증가한 것으로 판단
# #                     drug_hx.loc[i, 'subsetTrue'] = False
# #                     drug_hx.loc[i, 'rgmline'] = current_regimen_line + 1
# #                     drugnames_shown.update(current_drug)
# #                     current_regimen_line += 1
# #                 else :
# #                     # 두 조건을 충족하지 않으면 regimen은 유지
# #                     drug_hx.loc[i, 'rgmline'] = current_regimen_line
    
#         regimen_start_dates = sorted(drug_rgm_hx[drug_rgm_hx['subsetTrue']==False]['date'].tolist())
#         # 이 환자 투약정보에 의하면 두 regimen 사이의 최소 시각 간격 추정
#         # 최소시각간격 정의: 이전 regimen 마지막 투약시점부터 이번 regimen 최초 투약시점까지 시각간격
#         #least_tgap_btw_regimen = get_least_t_gap_btw_regimens(drug_hx)    

#         # 같은 항암제가 서로 다른 regimen에 속하는 것이 있을 때 각 항암제마다 최소시간각격 추정
#         #tgap_drug_duo_regimen_dict = get_least_t_gap_of_drug_in_two_regimens(drug_hx)    
#         #print(regimen_start_dates)
            
#     #민간도 분석을 해야할 free hyper-parameters
#     # 1. 같은 약을 투여할 때 가장 이웃한 투약기록과의 간격이 90일 이상이면 별도의 regimen에서 
#     #   투여한 약이라고 간주 ==> 90일?
#     # 2. regimen_matrix dataframe의 마지막 열인 'n_used_drugs'의 island를 찾는데
#     #   이 때 10일 이상 차이가 날때만 regimen이 끝나고 다른 regimen이 시작한 것으로 간주
    
#     # 환자의 항암제 투여기록, 수술기록, 병리기록을 분석하기 좋게 전처리가공
#     # Sorting pid_drug_chemo by drug_exposure_start_date
#     # Drug preprocessing
#     pid_drug_chemo.sort_values(by='drug_exposure_start_date', inplace=True)
#     #print(pid_drug_chemo)
#     # Making df, "regimen_matrix" from drug begining day to terminal day 
#     # 투여약들의 histories에서 최초항암제 투여일과 마지막 항암제 투여일을 검색
#     pid_drug_start_date = pid_drug_chemo['drug_exposure_start_date'].min()
#     pid_drug_end_date = pid_drug_chemo['drug_exposure_start_date'].max()
#     # 각 약별 (column에 들어갈 것들) 로 복용기간을 정의할 일자별 DateIndex를 DF로 만들어 준비
#     regimen_matrix = pd.date_range(start=pid_drug_start_date, 
#                          end=pid_drug_end_date).to_frame(name='dummy')
    
#     # unique drug name 
#     drug_names = pid_drug_chemo.loc[:, 'name'].drop_duplicates().values
#     #print(drug_names)
#     #print(regimen_matrix.columns)
#     # drug_names이 비어 있거나 한 개만 있는데 cisplatin, mitomycin C 일 경우엔 
#     # 0을 반환
#     if drug_names.shape[0] == 0 :
#         return 0
#     if len(drug_names) == 1 :
#         if re.match('cisplatin|mitomycin\nc', drug_names[0]) :
#             return 0

#     # drug_names.shape[0] > 1 일 때
#     #print('N. drugs:', drug_names.shape[0])
#     # 약의 종류가 2가지 이상일 때 regimen은 정하는 과정의 시작
#     for i, drug_name in enumerate(drug_names) : 
#         dates_drug = np.sort(pid_drug_chemo[pid_drug_chemo['name']==drug_name]\
#                                           .loc[:, ('drug_exposure_start_date')].values)
#         # 한 개의 약 투여하는 간격을 결정 : i번째와 i+1번쨰 항암제 투약 간격을 구함
#         intervals_drug = [(dates_drug[j+1]-dates_drug[j]).days for j in range(len(dates_drug)-1)]
#         # Checking a drug interval is whether too long or not
#         # 투여 간격이 3달 이상 차이가 나는 것이 있는 점검해서 있다면 별개의 약으로 봐야함
        
#         indices_other_interval = is_long_interval(intervals_drug)
#         # split a drug by more than one block
#         # 투약 간격이 긴 것이 있어서 한 약을 별개의 약으로 봐야할 경우 
#         if len(indices_other_interval) > 0 :
#             cnt = 1
#             j_prev = 0
#             for index_other_interval in indices_other_interval : 
#                 # intervals_drug는 dates_drug 간의 차이므로 
#                 # 실제 date_drug가 크게 변하는 index는 interval index에서 
#                 # 1을 더해 주어야 함
#                 j = index_other_interval + 1
#                 # 약의 투여기간 날짜를 칼럼으로 Join해서 붙임
#                 regimen_matrix = regimen_matrix.join(pd.date_range(start=dates_drug[j_prev:j][0], 
#                                                 end=dates_drug[j_prev:j][-1])\
#     https://mail.google.com/mail/u/0/#inbox/FMfcgxwLtQWrpDpWvqDTztdDxzZZmLbV                                            .to_frame(name=drug_name+str(cnt)))
#                 j_prev = j
#                 cnt += 1
#             #약의 투여기간 날짜를 칼럼으로 Join해서 붙임
#             regimen_matrix = regimen_matrix.join(pd.date_range(start=dates_drug[j:][0], 
#                                             end=dates_drug[j:][-1])\
#                                             .to_frame(name=drug_name+str(cnt)))
#         # 투약 간격들이 별개의 약으로 볼 만큼 길지 않아서 한 개의 약으로 봐도 될 경우 
#         else :
#             regimen_matrix = regimen_matrix.join(pd.date_range(start = dates_drug[0], 
#                                             end = dates_drug[-1]).to_frame(name=drug_name))
#     # 날짜가 들어있는 칼럼 삭제
#     regimen_matrix.drop(columns=['dummy'], inplace=True)
#     # 각 약들의 복용기간 중이며 해당 날짜가 적혀 있고 그렇지 않는 날이면 NaT가 저장되어 있는데 
#     #  이를 0 (NaT) 과 1로 변환
    
#     regimen_matrix = regimen_matrix.fillna(0).applymap(lambda x: 0 if x == 0 else 1)
#     regimen_matrix['used_n_drugs'] = regimen_matrix.sum(axis=1)
    
#     regimen_matrix = regimen_matrix.reset_index().rename(columns={'index':'date'})
#     regimen_matrix['date'] = pd.to_datetime(regimen_matrix['date'])
    
    
    
#     # regimen들 사이의 간격이 특정일 (현재 10일) 이상일 때만 별도의 regimen으로 판단함
    
#     btw_regimens = drop_tooshort_timespan_for_btw_regimens(\
#                                    zero_runs(regimen_matrix['used_n_drugs'].values))
#     #print("btw_regimens", btw_regimens)
#     # regimen_matrix에 regimen column을 만들어서 행이 regimen인지 아닌지 파악
#     # regimen이 한 개만 없을 때는 used_n_drugs column을 0과 1로만 변환
#     if len(btw_regimens)==0 :
#         regimen_matrix['rgm'] = regimen_matrix['used_n_drugs']\
#                                               .apply(lambda x: 1 if x>0 else 0)
#     #regimen이 2개 이상일 때
#     else :
#         rgm_column_value = np.zeros(regimen_matrix.shape[0])
#         rgm_bgn_index = 0
#         rgm_ter_index = regimen_matrix.shape[0]-1
#         for i, btw_regimen in enumerate(btw_regimens) :
#             if btw_regimen[0] != rgm_bgn_index :
#                 for j in range(rgm_bgn_index, btw_regimen[0], 1) :
#                     rgm_column_value[j] += i+1
            
#             rgm_bgn_index = btw_regimen[1]
#         while rgm_bgn_index < regimen_matrix.shape[0] :
#             rgm_column_value[rgm_bgn_index] += i+2
#             rgm_bgn_index += 1
#         regimen_matrix['rgm'] = rgm_column_value
#     #print(regimen_matrix[regimen_matrix['rgm']!=0])
    
#     #print(regimen_matrix[regimen_matrix['rgm']!=0])
#     # regimen 중에서 'cisplatin', 'mitomycin c' 단독으로 사용된 regimen이 있다면 없애자. 
#     drugs_adjuvant_head = ['cisplatin', 'mitomycin_c']
#     # regimem_matrix column names 중에 항암제만 남기고 삭제
#     drugs_regimen_matrix = regimen_matrix.columns
#     drugs_regimen_matrix = drugs_regimen_matrix[(drugs_regimen_matrix!='date')&\
#                                                 (drugs_regimen_matrix!='used_n_drugs')&\
#                                                 (drugs_regimen_matrix!='rgm')]
#     # cisplatin, mitomycin_c와 패턴 매칭이 되는 항암제가 있는지 찾아서 drugs_adjuvant에 입력
#     drugs_adjuvant = []
#     for drug_adjuvant_head in drugs_adjuvant_head :
#         r = re.compile(drug_adjuvant_head)
#         vmatch = np.vectorize(lambda x:bool(r.match(x)))
#         matched_arr = drugs_regimen_matrix[vmatch(drugs_regimen_matrix)]
#         matched_list = [x for x in matched_arr]
#         drugs_adjuvant += matched_list
# #         if re.search
#     for drug_adjuvant in drugs_adjuvant :
# #         if drug_adjuvant in  :
#             # regimen_matrix 중 rgm이 0이 아니고 used_n_drugs가 1이며, column 이름 중 
#             # drug_adjuvant의 약이름 컬럼 값이 1인 rgm의 유일한 값들을 찾고
#             rgms_arr = regimen_matrix[(regimen_matrix['rgm']!=0) & 
#                                            (regimen_matrix['used_n_drugs']==1) &
#                                            (regimen_matrix[drug_adjuvant]==1)]['rgm']\
#                                            .unique()
#             # 찾아진 regimen line을 하나씩 넣어서 각 line에 해당하는 행 중 used_n_drugs 값의 
#             # 최대값이 1과 같다면 drug_adjuvant가 단독으로 사용되었다고 할 수 있으며 만약 2이상의 값이
#             # 라면 drug_adjuvant와 다른 약도 사용한 경우가 있기 때문에 이 경우는 adjuvant가 아니다
#             for _rgm in rgms_arr :
#                 if regimen_matrix[regimen_matrix['rgm']==_rgm]['used_n_drugs'].max() > 1:
#                     continue
#                 else :
#                     # rgm이 _rgm0 값은 다시 0으로 되돌려 놓는다. 
#                     regimen_matrix.loc[:,'rgm'] = regimen_matrix['rgm']\
#                                                    .apply(lambda x: 0 if x==_rgm else x)                
#     #print(regimen_matrix[regimen_matrix['rgm']!=0])    
    
    
    
#     # patho preprocessing
#     if pid_patho.shape[0] > 0 :
#         pid_patho['procedure_date'] = pd.to_datetime(pid_patho['procedure_date'])
#         #pathos = pid_patho['procedure_date'].astype('datetime64[ns]')\
#         #                                     .to_frame(name='patho')
#         pathos = pid_patho['procedure_date'].to_frame(name='patho')
#         pathos.loc[:, 'date'] = pd.to_datetime(pathos['patho'])
#         # patho 정보를 regimen matrix df에 결합
#         regimen_matrix = pd.merge(regimen_matrix, pathos,
#                                   how='outer', on='date').fillna(0)
#         regimen_matrix.sort_values(by='date', inplace=True)
#         regimen_matrix.loc[:, 'patho'] = regimen_matrix['patho']\
#                                                .apply(lambda x: 0 if x == 0 else 1)
        
#         for i, row in pid_patho.iterrows() :

#             is_in_the_middle_of_regimen = \
#                     regimen_matrix[regimen_matrix['date'] == row['procedure_date']]['rgm'].values
#             pre_drug_hx_patho = regimen_matrix[regimen_matrix['date'] < row['procedure_date']]
#             if len(pre_drug_hx_patho[pre_drug_hx_patho['rgm']!=0])==0 :
#                 pre_drug_hx_patho = pd.DataFrame()
#             post_drug_hx_patho = regimen_matrix[regimen_matrix['date'] > row['procedure_date']]
#             if len(post_drug_hx_patho[post_drug_hx_patho['rgm']!=0])==0 :
#                 post_drug_hx_patho = pd.DataFrame()
#             # patho가 regimen 속에 있다면 patho를 중요한 Event로 보지 않고 
#             # patho 전 후에 adjuvant나 neoadjuvant를 판단할 필요가 없음  필요없음
#             if len(is_in_the_middle_of_regimen)==1 :
#                 if is_in_the_middle_of_regimen > 0 :
#                     continue
#                 else :
#                     # patho 앞쪽 drug 투약기록으로부터 병리검사일 이전 가장 가까운 투약기록을 찾고 
#                     # 그 시간 차이를 구해서 8주 이내면 neoadjuvant로 보고 regimen에서 제거
#                     if pre_drug_hx_patho.shape[0] > 0 :
#                         # 병리검사 앞쪽 regimen이 한 개 뿐이라면
#                         if pre_drug_hx_patho['rgm'].max()==1 :
#                             rgm_line = 1
#                         # 병리검사 앞쪽 regimen이 두 개 이상이며, 그 중 병리검사일시에서 가까운쪽 regimen의 
#                         # 마지막 투약시기가 8주 이내인가? 
#                         else :
#                             pre_closest_drug_date = \
#                                 pre_drug_hx_patho[pre_drug_hx_patho['rgm']!=0]['date'].max()
#                             if (row['procedure_date'] - pre_closest_drug_date).days < adjuvant_limit :
#                                 rgm_line = pre_drug_hx_patho[pre_drug_hx_patho['date']==\
#                                                               pre_closest_drug_date]['rgm'].iloc[0]
#                             else :
#                                 rgm_line = None
#                         # 병리검사일 이전에 regimen이 1개이거나 2개 이상 일때 가장 근접한 과거의 투약기록이 
#                         # 8주 이내일 때 regimen을 neoadjuvant로 간주
#                         #print("pre rgm_line:", rgm_line)
#                         if rgm_line != None :
#                             regimen_matrix.loc[:,'rgm'] = regimen_matrix['rgm']\
#                                                              .apply(lambda x: 0 if x==rgm_line else x)
#                             #print('pre:',pre_closest_drug_date, rgm_line)
#                     # biopst 뒷쪽 drug 투약기록으로부터 병리검사일 이후 가장 가까운 투약기록을 찾고
#                     # 그 시간 차이가 8주 이내면 adjuvant로 보고 regimen에서 제거
#                     if post_drug_hx_patho.shape[0] > 0 :
#                         post_closest_drug_date = \
#                             post_drug_hx_patho[post_drug_hx_patho['rgm']!=0]['date'].min()
#                         if (post_closest_drug_date - row['procedure_date']).days < adjuvant_limit :
#                             rgm_line = post_drug_hx_patho[post_drug_hx_patho['date']==\
#                                                            post_closest_drug_date]['rgm'].iloc[0]
#                         else :
#                             rgm_line = None
#                         #print("post rgm_line:", rgm_line)
#                         if rgm_line != None :
#                             regimen_matrix.loc[:, 'rgm'] = regimen_matrix['rgm']\
#                                                         .apply(lambda x: 0 if x==rgm_line else x)
    
#     #print(regimen_start_dates)
#     print(regimen_matrix[regimen_matrix['rgm']!=0])
#     if 'regimen_start_dates' in locals():
#         #print(regimen_matrix.tail())
#         #print(np.flatnonzero(regimen_matrix['date'].isin(regimen_start_dates[1:])))
#         #print(regimen_matrix[regimen_matrix['rgm']!=0]\
#         #      .iloc[np.flatnonzero(regimen_matrix[regimen_matrix['rgm']!=0]['date']\
#         #                           .isin(regimen_start_dates[1:]))][['date', 'rgm']])
#         # number of regimens should not be larger than 2 * N(drugs)
#         #if drug_names.shape[0] * 2 > len(regimen_start_dates) :
#         for regimen_start_date in regimen_start_dates[1:] : 
#             regimen_matrix['rgm_diff'] = regimen_matrix['rgm'].diff()
#             disagreed = regimen_matrix[(regimen_matrix['rgm']!=0) & \
#                                        (regimen_matrix['rgm_diff']==0)]\
#                             .iloc[np.flatnonzero(regimen_matrix[(regimen_matrix['rgm']!=0) &\
#                                                                 (regimen_matrix['rgm_diff']==0)]['date']\
#                                 .eq(regimen_start_date))]

#             if disagreed.shape[0] > 0 :
#                 regimen_matrix['rgm'] = regimen_matrix.apply(lambda x: x['rgm']+1 \
#                                                              if x['date']>=regimen_start_date and\
#                                                                x['rgm'] > 0 else x['rgm'], axis=1)
#     #                 bgn_index = np.flatnonzero(regimen_matrix['date'].eq(regimen_start_date))
#     #                 print(bgn_index)
#     #                 print (regimen_matrix.loc[bgn_index[0]-2:bgn_index[0]+2,
#     #                                           ('date', 'rgm', 'rgm_diff')])
# #         else :
# #             print("number of regimen way to much ")
#     # 무엇 때문에 regimen의 개수가 약제의 개수+1보다 한 개 더 많을 때 다시 계산해야 한다고 생각한 근거는 없다. 
#     # drug_hx를 weekly resampling 한 후에 다시 regimen을 계산해 보자 
# #     if regimen_matrix['rgm'].max() > len(drug_names):
# #         row_union = lambda x: reduce(set.union, x.values, set({}))
# #         drug_hx_weekly = drug_hx.set_index('date')\
# #                             .resample('W').agg({'name':row_union})
# #         drug_hx_weekly = drug_hx_weekly[drug_hx_weekly['name']!=set({})].reset_index()
# #         drug_rgm_hx_weekly = regimen_by_subset_drugs(drug_hx_weekly)     
# #         regimen_start_dates = sorted(drug_rgm_hx_weekly[drug_rgm_hx_weekly['subsetTrue']==\
# #                                                         False]['date'].tolist())
# #         print(drug_rgm_hx_weekly)
# #         print(regimen_start_dates)
#     regimen_matrix.loc[:, 'date'] = regimen_matrix['date'].dt.date
#     return regimen_matrix

In [ ]:
# def get_least_t_gap_of_drug_in_two_regimens(drug_hx) :    
    
#     ######################################################################
#     # regimen에 어떤 항암제를 투여했는지 찾고 각 regimen별 항암제 종류를 사전으로 제작
#     # 같은 이름의 항암제가 서로 다른 regimen에 포함되었다면 그 최소 시각 간격 결정
#     # input :  drug_hx (pd.DataFrame)
#     # output:  min_tgap_drug_two_regimens (dictionary[항암제]=최소시간간격)
#     ######################################################################
    
#     # 합집합 연산 함수 정의
#     my_lambda = lambda x: reduce(set.union, x)
#     # 같은 regimen에 속한 모든 항암제의 합집합을 구해 name 칼럼 입력
#     drugs_in_regimen = drug_hx.groupby('rgmline').aggregate({'name': my_lambda})
#     # df를 dict으로 변환                                                        
#     drugs_in_regimen_dict = drugs_in_regimen['name'].to_dict()
#     # 각 key당 values의 element (항암제이름) 하나씩 뽑아서 모든 조합을 항암제튜플로 제작 후 
#     # 튜플의 항암제 원소 중 중복된 항암제를 list의 하나씩 추가
#     drugs_in_multiple_regimen = []
#     for comb in list(product(*drugs_in_regimen_dict.values())) :
#         for elem in comb :
#             if comb.count(elem) > 1 :
#                 drugs_in_multiple_regimen.append(elem)
#     # 역시 모든 조합에서 중복된 항암제를 찾다보니 리스트 안에도 중복된 항암제 이름이 있어서 또 중복 제거
#     # 이렇게 하면 2개 이상의 regimen에 들어간 항암제 리스트를 구성할 수 있음 
#     drugs_in_multiple_regimen = list(set(drugs_in_multiple_regimen))
    
#     min_tgap_drug_two_regimens={}
#     if len(drugs_in_multiple_regimen) == 0 :
#         return min_tgap_drug_two_regimens
    
#     # 같은 항암제가 서로 다른 regimen에 나오는 가장 짧은 시간 간격 검색
#     for drug_in_multiple_regimen in  drugs_in_multiple_regimen :
#         # drug_hx의 name 열에 항암제가 있는 열만 선택하여 별도의 df로 저장
#         drug_shown_rows_df = drug_hx[drug_hx['name']\
#                                     .map(lambda x: drug_in_multiple_regimen in x)]
#         # date, rgmline 칼럼들의 이웃한 두 행의 차이를 계산하여 저장 (date의 unit day로)
#         drug_shown_rows_df['rgm_diff'] = drug_shown_rows_df['rgmline'].diff()
#         drug_shown_rows_df['day_diff'] = \
#                             drug_shown_rows_df['date'].diff()/np.timedelta64(1,'D')
#         # 같은 항암제가 다른 regimen에 속하는 경우 최소한의 시간 간격 결정 
#         min_tgap_drug_in_multiple_regimen = \
#             drug_shown_rows_df[drug_shown_rows_df['rgm_diff']>0]['day_diff'].min()
#         # 항암제 이름을 key로 최소 시간 간격 유리수를 value로 사전에 저장
#         min_tgap_drug_two_regimens[drug_in_multiple_regimen] = \
#                                                   min_tgap_drug_in_multiple_regimen
#         # 사전을 반환
#     return min_tgap_drug_two_regimens

In [ ]:
# def get_least_t_gap_btw_regimens(drug_hx) :
    
#     ####################################################
#     # regimen 사이 최소 시간 간격 결정
#     # input :  drug_hx (pd.DataFrame)
#     # output:  min_btw_regimen_tgap (float, unit: days)
#     ####################################################
    
#     # drug_hx의 subsetTrue 열이 False 행은 regimen이 달라지는 시작시점
#     regimen_start_dates = \
#         sorted(drug_hx[drug_hx['subsetTrue']==False]['date'].tolist())
#     # 그 바로 한 행 전이 이전 regimen의 마지막 투약시점으로 regimen 종료시점
#     regimen_end_dates = \
#         sorted(drug_hx.iloc[np.flatnonzero(drug_hx['subsetTrue']==False)-1]['date'].tolist())
#     # 만약 regimen first line만 있다면 min_btw_regimen_tgap = None이 됨
#     # regimen이 2개 이상 있는 경우 
#     if len(regimen_start_dates) == len(regimen_end_dates) and \
#       len(regimen_start_dates) > 1 :
#         # 이 환자에 대한 regimen 최소 시간 간격 결정
#         btw_regimen_tgap = []
#         for i in range(len(regimen_end_dates)-1) :
#             btw_regimen_tgap.append((regimen_start_dates[i+1] - regimen_end_dates[i]).days)
#         min_btw_regimen_tgap = min(btw_regimen_tgap)
#     else :
#         # regimen이 1개만 있는 경우
#         min_btw_regimen_tgap = None
#     # regimen 간격 중 가장 짧은 시각 간격을 반환
#     return min_btw_regimen_tgap

In [ ]:
print(personid_cancertypes_df.shape)
personid_cancertypes_df.to_csv("chemos.csv", index=False)

In [ ]:
fig, ax = plt.subplots(1, 1)
ndrugs = pd.read_csv("patient_drug_numbers.txt", header=None)
ndrugs.columns = ['pid', 'ndrug']
ndrugs['ndrug'].hist(bins=15, grid=True, color='tab:orange',
                     xlabelsize=14, ylabelsize=14, 
                     figsize=(8.9, 4.3), legend=False, ax=ax)
ax.set_xlabel("Number of types of anti-cancer drugs prescribed by a patient", fontsize=14)
ax.set_ylabel("Number of Patients", fontsize=14)
ax.set_title("Histogram of the types of anticancer drugs for the Chemo patient group", fontsize=14)

In [ ]:
sys.exit(1)

In [ ]:
# # Program to count islands in boolean 2D matrix 
# class Graph: 
  
#     def __init__(self, row, col, g): 
#         self.ROW = row 
#         self.COL = col 
#         self.graph = g 
  
#     # A function to check if a given cell  
#     # (row, col) can be included in DFS 
#     def isSafe(self, i, j, visited): 
#         # row number is in range, column number 
#         # is in range and value is 1  
#         # and not yet visited 
#         return (i >= 0 and i < self.ROW and 
#                 j >= 0 and j < self.COL and 
#                 not visited[i][j] and self.graph[i][j]) 
              
  
#     # A utility function to do DFS for a 2D  
#     # boolean matrix. It only considers 
#     # the 8 neighbours as adjacent vertices 
#     def DFS(self, i, j, visited): 
  
#         # These arrays are used to get row and  
#         # column numbers of 8 neighbours  
#         # of a given cell 
#         rowNbr = [-1, -1, -1,  0, 0,  1, 1, 1]; 
#         colNbr = [-1,  0,  1, -1, 1, -1, 0, 1]; 
          
#         # Mark this cell as visited 
#         visited[i][j] = True
  
#         # Recur for all connected neighbours 
#         for k in range(8): 
#             if self.isSafe(i + rowNbr[k], j + colNbr[k], visited): 
#                 self.DFS(i + rowNbr[k], j + colNbr[k], visited) 
  
  
#     # The main function that returns 
#     # count of islands in a given boolean 
#     # 2D matrix 
#     def countIslands(self): 
#         # Make a bool array to mark visited cells. 
#         # Initially all cells are unvisited 
#         visited = [[False for j in range(self.COL)]for i in range(self.ROW)] 
  
#         # Initialize count as 0 and travese  
#         # through the all cells of 
#         # given matrix 
#         count = 0
#         for i in range(self.ROW): 
#             for j in range(self.COL): 
#                 # If a cell with value 1 is not visited yet,  
#                 # then new island found 
#                 if visited[i][j] == False and self.graph[i][j] ==1: 
#                     # Visit all cells in this island  
#                     # and increment island count 
#                     self.DFS(i, j, visited) 
#                     count += 1
  
#         return count 

In [ ]:
# def category_segmentation(x) :
#     boundaries = []
#     for i in range(len(x)) :
#         front = set(x[:i+1])
#         back = set(x[(i+1):])
#         if front.intersection(back) == set() :
#             if i == len(x) - 1:
#                 boundaries.append(0)
#             else :
#                 boundaries.append(1)
        
#         else :
#             boundaries.append(0)
#     return boundaries

In [ ]:
# def goodness_of_variance_fit(df) :
#     columns_ldjenks = [x for x in df.columns if re.search(r'ldjenks', x)]
#     array = df['lapse_days']
#     sdam = np.sum((array - array.mean()) ** 2)
#     total = [1]
#     for col in columns_ldjenks : 
#         sdcm = 0
#         for j in df[col].astype('category').unique().to_list() :
#             arr = df[df[col] == j]['lapse_days'].values
#             sdcm += np.sum((arr - arr.mean())**2)
#         total.append((sdam - sdcm) / sdam)
#     return total

In [ ]:
# def match_drugtype_boundaries(df) :
#     columns_ldjenks = [x for x in df.columns if re.search(r'ldjenks', x)]
#     disagreements = [0]
#     for col in columns_ldjenks : 
#         disgreement = 0
#         for j in range(df.shape[0]-1):
#             if df.loc[j, col] != df.loc[j+1, col] :
#                 if df.loc[j, 'boundary'] != 1 :
#                     disgreement += 1
#         disagreements.append(disgreement)
#     return disagreements

In [ ]:
# def decision_regimen(df) :
    
#     df.index = np.arange(len(df))
#     #print(df)
#     drug_types = df['drug_factor'].values
#     boundaries = category_segmentation(drug_types)
#     df['boundary'] = boundaries
#     gvf = np.array(goodness_of_variance_fit(df))
#     disagreements = np.array(match_drugtype_boundaries(df))
#     for group in np.argsort(gvf) :
#         if disagreements[group] == 0 :
#             return group+1
#     return None

In [ ]:
# estimated_regimen = pd.DataFrame(columns = ['person_id', 'Nth_regimen', 'isDead'])

# for i, pid in enumerate(tqdm(pid_chemo)) :
#     #print(pid)
#     # patient death information
#     isDead = death_pd[death_pd['person_id'] == pid].shape[0]
    
#     pid_drug_chemo_tmp = drug_df[drug_df['person_id']==pid]\
#                                 .loc[:, ('drug_exposure_start_date', 'drug_exposure_end_date',
#                                         'stop_reason', 'drug_source_value', 'refills', 
#                                         'quantity', 'days_supply', 'sig',
#                                         'dose_unit_source_value', 'code', 'name')]
#     pid_drug_chemo = pid_drug_chemo_tmp[pid_drug_chemo_tmp['drug_source_value']\
#                                        .isin(chemo_drug['code'])]
#     pid_drug_chemo = pid_drug_chemo.loc[:, ('drug_exposure_start_date', 'code', 'name')]\
#                                    .drop_duplicates()
    
#     # number of categorical variable for title
#     pid_drug_chemo.sort_values(by='drug_exposure_start_date', inplace=True)
#     pid_drug_chemo['drug_factor'] = pd.factorize(list(pid_drug_chemo['name'].values))[0]
    
#     drug_ts = pid_drug_chemo[['drug_exposure_start_date', 'drug_factor']]
#     # pandas drug_ts onehotcode
#     #drug_ts = pd.concat([drug_ts,pd.get_dummies(drug_ts['drug_factor'])],axis=1)
#     firstday_anticacner_drug = drug_ts['drug_exposure_start_date'].min()
#     drug_ts['lapse_days'] = drug_ts['drug_exposure_start_date']\
#                            .apply(lambda x: (x - firstday_anticacner_drug).days)
#     # 화학항암제가 한 가지 종류라면 무조건 regimen은 1개
# #     if pid == 1358 :
# #         print(drug_ts)
#     if len(drug_ts['drug_factor'].unique()) == 1 :
#         estimated_regimen.loc[len(estimated_regimen)] = [pid, 1, isDead]
#         continue
#     # 항암제 투여 횟수가 2번이하거나 항암제 종류가 1개 이하면 무조건 regimen은 1
#     if drug_ts.shape[0] < 3 or len(drug_ts['drug_factor'].unique()) < 2:
#         estimated_regimen.loc[len(estimated_regimen)] = [pid, 1, isDead]
#     else :
#         breaks2 = jenkspy.jenks_breaks(drug_ts['lapse_days'], nb_class=2)
#         # breaks argument에 중복이 있다면 regimen은 1
#         if len(set(breaks2)) != len(breaks2) : 
#             estimated_regimen.loc[len(estimated_regimen)] = [pid, 1, isDead]
#             continue
#         drug_ts['ldjenks2'] = pd.cut(drug_ts['lapse_days'], bins=breaks2, 
#                                          labels=['0', '1'], include_lowest=True)
#         # 항암제 종류가 2개 이하면 
#         if len(drug_ts['drug_factor'].unique()) < 3 :
#             rgm = decision_regimen(drug_ts)
#             estimated_regimen.loc[len(estimated_regimen)] = [pid, rgm, isDead]
#         else :
#             breaks3 = jenkspy.jenks_breaks(drug_ts['lapse_days'], nb_class=3)
#             # break argument에 중복이 있다면 regimen은 2
#             if len(set(breaks3)) != len(breaks3):
#                 rgm = decision_regimen(drug_ts)
#                 estimated_regimen.loc[len(estimated_regimen)] = [pid, rgm, isDead]
#                 continue
#             drug_ts['ldjenks3'] = pd.cut(drug_ts['lapse_days'], bins=breaks3, 
#                                  labels=['0', '1', '2'],
#                                  include_lowest=True)
#             rgm = decision_regimen(drug_ts)
#             estimated_regimen.loc[len(estimated_regimen)] = [pid, rgm, isDead]

In [ ]:
# islands_chemo = pd.DataFrame()
# for i, pid in enumerate(pid_chemo) :
    
#     pid_drug_chemo_tmp = drug_df[drug_df['person_id']==pid]\
#                                 .loc[:, ('drug_exposure_start_date', 'drug_exposure_end_date',
#                                         'stop_reason', 'drug_source_value', 'refills', 
#                                         'quantity', 'days_supply', 'sig',
#                                         'dose_unit_source_value', 'code', 'name')]
#     pid_drug_chemo = pid_drug_chemo_tmp[pid_drug_chemo_tmp['drug_source_value']\
#                                        .isin(chemo_drug['code'])]
#     tmp_drug_chemo = pid_drug_chemo.loc[:, ('code', 'name', 'quantity', 
#                                             'dose_unit_source_value')]\
#                                    .drop_duplicates()
#     # title = easy identifier of drugs and dosages
#     tmp_drug_chemo['title'] = tmp_drug_chemo['name'].str.strip() + "\n"+tmp_drug_chemo['quantity']+\
#                                 tmp_drug_chemo['dose_unit_source_value']
#     tmp_drug_chemo['title'] = tmp_drug_chemo['title'].astype('category')
#     tmp_drug_chemo.drop(columns = ['name', 'quantity', 'dose_unit_source_value'], 
#                         inplace = True)
#     # number of categorical variable for title
#     tmp_drug_chemo['drug_factor'] = pd.factorize(list(tmp_drug_chemo['title'].values))[0]
    
#     pid_drug_chemo = pd.merge(pid_drug_chemo, tmp_drug_chemo, on='code')
#     drug_ts = pid_drug_chemo[['drug_exposure_start_date', 'drug_factor']]
#     drug_ts['start_date'] = pd.to_datetime(drug_ts['drug_exposure_start_date'])
#     drug_ts = drug_ts.sort_values(by='start_date')
#     # pandas df onehotcode
#     df = pd.concat([drug_ts,pd.get_dummies(drug_ts['drug_factor'])],axis=1)
#     df.drop(['drug_factor', 'drug_exposure_start_date'],axis=1, inplace=True)
#     df = df.set_index('start_date')
#     island_dict = {}
#     for k, interval in enumerate(['W', '2W', '3W', '4W', '5W', '6W', '7W', '8W',
#                                   '9W', '10W', '11W', '12W', '13W', '14W', '15W']) :
        
#         rdf = df.resample(interval).sum()
#         rdf = rdf.fillna(0).applymap(int)
#         rdf['s'] = rdf.apply(sum, axis=1)
#         runs = zero_runs(rdf['s'].to_numpy())
#         island_dict[interval] = runs.shape[0] + 1
#     islands_chemo = islands_chemo.append(pd.DataFrame(island_dict, index=[pid]))

#     if i == 20 :
#         print(islands_chemo)
#         sys.exit(-1)

In [ ]:
# islands_chemo = pd.DataFrame()
# for i, pid in enumerate(pid_chemo) :
    
#     pid_drug_chemo_tmp = drug_df[drug_df['person_id']==pid]\
#                                 .loc[:, ('drug_exposure_start_date', 'drug_exposure_end_date',
#                                         'stop_reason', 'drug_source_value', 'refills', 
#                                         'quantity', 'days_supply', 'sig',
#                                         'dose_unit_source_value', 'code', 'name')]
#     pid_drug_chemo = pid_drug_chemo_tmp[pid_drug_chemo_tmp['drug_source_value']\
#                                        .isin(chemo_drug['code'])]
#     tmp_drug_chemo = pid_drug_chemo.loc[:, ('code', 'name', 'quantity', 
#                                             'dose_unit_source_value')]\
#                                    .drop_duplicates()
#     # title = easy identifier of drugs and dosages
#     tmp_drug_chemo['title'] = tmp_drug_chemo['name'].str.strip() + "\n"+tmp_drug_chemo['quantity']+\
#                                 tmp_drug_chemo['dose_unit_source_value']
#     tmp_drug_chemo['title'] = tmp_drug_chemo['title'].astype('category')
#     tmp_drug_chemo.drop(columns = ['name', 'quantity', 'dose_unit_source_value'], 
#                         inplace = True)
#     # number of categorical variable for title
#     tmp_drug_chemo['drug_factor'] = pd.factorize(list(tmp_drug_chemo['title'].values))[0]
    
#     pid_drug_chemo = pd.merge(pid_drug_chemo, tmp_drug_chemo, on='code')
#     drug_ts = pid_drug_chemo[['drug_exposure_start_date', 'drug_factor']]
#     drug_ts['start_date'] = pd.to_datetime(drug_ts['drug_exposure_start_date'])
#     drug_ts = drug_ts.sort_values(by='start_date')
#     # pandas df onehotcode
#     df = pd.concat([drug_ts,pd.get_dummies(drug_ts['drug_factor'])],axis=1)
#     df.drop(['drug_factor', 'drug_exposure_start_date'],axis=1, inplace=True)
#     df = df.set_index('start_date')
#     island_dict = {}
#     for k, interval in enumerate(['W', '2W', '3W', '4W', '5W', '6W', '7W', '8W']) :
#         for j, col in enumerate(df.columns) :
#             if j == 0  :
#                 rdf = df.resample(interval)[col].sum().to_frame()
#             else :
#                 rdf = pd.concat([rdf, df.resample(interval)[col].sum()], axis=1)
#         rdf = rdf.fillna(0).applymap(int)
#         rdf = rdf.applymap(lambda x: 1 if x > 0 else 0)
#         graph = rdf.to_numpy()
#         g = Graph(len(graph), len(graph[0]), graph)
#         island_dict[interval] = g.countIslands()
#     #print(interval, g.countIslands())
#     islands_chemo = islands_chemo.append(pd.DataFrame(island_dict, index=[pid]))
#     print(islands_chemo)
        
# #     # now drop the original 'country' column (you don't need it anymore)
# #     df.drop(['drug_factor', 'drug_exposure_start_date'],axis=1, inplace=True)
# #     print(df.head())
# #     print(df.set_index('start_date').resample('M').applymap(func))
# #     graph = [[1, 1, 0, 0, 0], 
# #             [0, 1, 0, 0, 1], 
# #             [1, 0, 0, 1, 1], 
# #             [0, 0, 0, 0, 0], 
# #             [1, 0, 1, 0, 1]] 


# #     row = len(graph) 
# #     col = len(graph[0]) 

# #     g= Graph(row, col, graph) 

# #     print "Number of islands is:"
# #     print g.countIslands() 
  
# #This code is contributed by Neelam Yadav 
#     if i > 10 :
#         sys.exit(-1)